# Práctica 4 - Competición de Kaggle  
Lucía Pazos Santiago, Anxo Trillo Santamaría, Javier Ares Vila

En primer lugar, se importarán las librerías necesarias para el desarrollo de esta práctica. Si no están ya instaladas, será necesario instalarlas para poder usarlas.

In [ ]:
%pip install scikit-learn
%pip install imblearn
%pip install pandas
%pip install matplotlib
%pip install tdqm
%pip install xgboost
%pip install lightgbm
%pip install catboost --prefer-binary
%pip install tensorflow
%pip install keras
%pip install scikeras

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Using cached graphviz-0.20.3-py3-none-any.whl.metadata (12 kB)
  Using cached plotly-6.0.1-py3-none-any.whl.metadata (6.7 kB)
  Using cached narwhals-1.38.0-py3-none-any.whl.metadata (9.3 kB)
   ---------------------------------------- 0.0/102.5 MB ? eta -:--:--
   ---------------------------------------- 0.8/102.5 MB 6.7 MB/s eta 0:00:16
   - -------------------------------------- 3.4/102.5 MB 11.8 MB/s eta 0:00:09
   -- ------------------------------------- 5.2/102.5 MB 10.3 MB/s eta 0:00:10
   -- -------------------------------

In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import imblearn

from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import *
from sklearn.decomposition import *
from sklearn.feature_selection import *
from sklearn.model_selection import *
from sklearn.metrics import *
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from imblearn.under_sampling import RandomUnderSampler
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.linear_model import Ridge
from sklearn.svm import SVR
from tqdm import tqdm
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.base import clone
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import GradientBoostingRegressor

## Introducción

En esta práctica desarrollaremos un modelo predictivo que permita estimar el precio de viviendas en Galicia a partir de diferentes características disponibles en un conjunto de datos proporcionados por una competición de Kaggle (https://www.kaggle.com/competitions/predicion-do-prezo-de-vivendas/overview). 

Para ello, llevaremos a cabo un análisis de los datos con el fin de entender su estructura y encontrar posibles problemas. A continuación, aplicaremos técnicas de preprocesamiento, seleccionaremos y entrenaremos distintos modelos y ajustaremos sus hiperparámetros. Por último generaremos predicciones sobre el conjunto de test, las cuales serán enviadas a Kaggle, buscando maximizar la percisión.

## Descripción de los datos

En esta sección se describen las principales características del conjunto de datos. Para ello, se carga el dataset de entrenamiento desde un archivo CSV y se muestra mediante display(df1). Como se puede observar, el dataset está compuesto por 20000 muestras y 20 columnas, entre las cuales se encuentran tanto variables numeréricas como categóricas. 

Cabe destacar que a simple vista ya se puede observar como algunas columnas presentan valores NaN.

In [25]:
df1 = pd.read_csv('./data/train.csv', delimiter=',')
display(df1)

,id,superficie_interior_m2,superficie_exterior_m2,numero_habitacions,numero_banos,ano_construccion,lonxitude,latitude,temperatura_media_mes_construccion,tipo_edificacion,calidade_materiais,cor_favorita_propietario,distancia_centro_km,distancia_escola_km,indice_criminalidade,acceso_transporte_publico,orientacion,eficiencia_enerxetica,numero_arboles_xardin,prezo_euros
0,25521,44.96,0.00,1,2,1947,-8.17,43.20,24.75,Apartamento,Media,Azul,23.68,2.75,0.35,Bo,Norte,C,0,51938
1,4843,93.55,87.26,2,2,1977,-7.23,43.60,14.06,Apartamento,Alta,Verde,NaN,16.67,0.44,Bo,Este,C,0,76891
2,27734,273.64,187.29,1,1,1996,-8.40,42.25,12.27,Chalet adosado,Media,Branco,29.44,18.96,0.47,Regular,Este,C,10,261441
3,22142,171.82,54.89,4,2,1996,-6.81,43.15,11.61,Chalet adosado,Alta,Vermello,53.31,1.96,0.35,Bo,Este,G,3,247821
4,14748,NaN,2153.49,1,1,1990,-8.76,42.92,10.04,Casa,Baixa,Verde,70.20,4.72,0.09,Bo,Oeste,G,12,400645
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,29803,122.77,2285.95,6,2,2003,-8.34,43.63,5.74,Casa,Alta,Branco,65.57,12.38,0.69,Bo,Oeste,C,0,443729
19996,5391,62.59,33.16,2,2,1970,-6.77,42.62,14.61,Apartamento,Media,Amarelo,21.46,21.57,0.15,Bo,Norte,C,0,50000
19997,861,144.47,427.37,4,1,1988,-6.87,42.00,14.74,Casa,Media,Vermello,NaN,0.72,NaN,Regular,Este,B,6,223467
19998,15796,139.52,0.00,6,2,2014,-7.21,43.38,18.15,Apartamento,Alta,Azul,15.45,14.61,0.28,Bo,Oeste,B,0,186403


In [26]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 20 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   id                                  20000 non-null  int64  
 1   superficie_interior_m2              18993 non-null  float64
 2   superficie_exterior_m2              18973 non-null  float64
 3   numero_habitacions                  20000 non-null  int64  
 4   numero_banos                        20000 non-null  int64  
 5   ano_construccion                    20000 non-null  int64  
 6   lonxitude                           20000 non-null  float64
 7   latitude                            20000 non-null  float64
 8   temperatura_media_mes_construccion  20000 non-null  float64
 9   tipo_edificacion                    20000 non-null  object 
 10  calidade_materiais                  20000 non-null  object 
 11  cor_favorita_propietario            20000

## Preprocesado (Redactar cuando se haya escogido qué aplicar)

A continuación, se lleva a cabo el preprocesado del conjunto de datos con el objetivo de prepararlo para la creación del modelo asegurando así que aprenda de una forma más eficiente y precisa. Este proceso incluye la gestión de valores ausentes, la conversión de variables categóricas a numéricas, la detección de outliers, la normalización de variables...

### Valores faltantes

Como se comprobó en el apartado anterior al analizar el dataset, varias columnas contienen valores nulos. Más especificamente superficie_interior_m2, superficie_exterior_m2, distancia_centro_km, distancia_escola_km e indice_criminalidade, por lo que será necesario gestionarlos.

Se han probado varias opciones como eliminar directamente las filas con valores nulos o sustituir los nulos por el valor de la media de la columna, pero se ha observado que los mejores resultados han sido obtenidos sustituyendo los valores faltantes por la mediana.

In [27]:
#df1 = df1.dropna() # Opción de liminar filas con valores NaN
#df1 = df1.fillna(df1.mean(numeric_only=True)) # Opción de rellenar NaN con la media de la columna
df1 = df1.fillna(df1.median(numeric_only=True))
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 20 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   id                                  20000 non-null  int64  
 1   superficie_interior_m2              20000 non-null  float64
 2   superficie_exterior_m2              20000 non-null  float64
 3   numero_habitacions                  20000 non-null  int64  
 4   numero_banos                        20000 non-null  int64  
 5   ano_construccion                    20000 non-null  int64  
 6   lonxitude                           20000 non-null  float64
 7   latitude                            20000 non-null  float64
 8   temperatura_media_mes_construccion  20000 non-null  float64
 9   tipo_edificacion                    20000 non-null  object 
 10  calidade_materiais                  20000 non-null  object 
 11  cor_favorita_propietario            20000

### Valores duplicados

Para encontrar los valores duplicados utilizaremos la función drop_duplicates() con una configuración que permite eliminar filas/columnas duplicadas manteniendo solo la primera fila/columna del conjunto de duplicados devolviendo un nuevo dataset.

In [28]:
# Eliminar filas duplicadas
df2 =df1.drop_duplicates(subset=None, keep='first', inplace=False)
print(len(df1))
print(len(df2))

20000
20000


### Gestión de atributos categóricos

In [29]:
# Eliminar filas con valores string
for elm in df2:
    if(isinstance(df2[elm].iloc[1],str)):
        #print(elm)
        df2=df2.drop(elm,axis=1)

df2 = df2.reset_index(drop=True)
display(df2)

,id,superficie_interior_m2,superficie_exterior_m2,numero_habitacions,numero_banos,ano_construccion,lonxitude,latitude,temperatura_media_mes_construccion,distancia_centro_km,distancia_escola_km,indice_criminalidade,numero_arboles_xardin,prezo_euros
0,25521,44.96,0.00,1,2,1947,-8.17,43.20,24.75,23.68,2.75,0.35,0,51938
1,4843,93.55,87.26,2,2,1977,-7.23,43.60,14.06,16.98,16.67,0.44,0,76891
2,27734,273.64,187.29,1,1,1996,-8.40,42.25,12.27,29.44,18.96,0.47,10,261441
3,22142,171.82,54.89,4,2,1996,-6.81,43.15,11.61,53.31,1.96,0.35,3,247821
4,14748,125.68,2153.49,1,1,1990,-8.76,42.92,10.04,70.20,4.72,0.09,12,400645
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,29803,122.77,2285.95,6,2,2003,-8.34,43.63,5.74,65.57,12.38,0.69,0,443729
19996,5391,62.59,33.16,2,2,1970,-6.77,42.62,14.61,21.46,21.57,0.15,0,50000
19997,861,144.47,427.37,4,1,1988,-6.87,42.00,14.74,16.98,0.72,0.33,6,223467
19998,15796,139.52,0.00,6,2,2014,-7.21,43.38,18.15,15.45,14.61,0.28,0,186403


In [30]:
# Pasar categorías a números

### Eliminación de columnas

(Añadir también lo de que se probó a seleccionar las características más relevantes pero se vio que no mejoraba resultados)

Por otro lado, también es conveniente eliminar las columnas que no aporten ningún valor predictivo al modelo, como es el caso de la columna id que simplemente actúa como identificador único de cada muestra.

In [31]:
df3 = df2.drop(columns=["id"])

También se separará la variable objetivo (prezo_euros) del resto de variables del dateset, obteniendo así un matriz X que contendrá únicamente las variables independientes que se emplearán en el entrenamiento.

In [32]:
target = df3["prezo_euros"]
X = df3.drop(columns=["prezo_euros"])
display(X)

,superficie_interior_m2,superficie_exterior_m2,numero_habitacions,numero_banos,ano_construccion,lonxitude,latitude,temperatura_media_mes_construccion,distancia_centro_km,distancia_escola_km,indice_criminalidade,numero_arboles_xardin
0,44.96,0.00,1,2,1947,-8.17,43.20,24.75,23.68,2.75,0.35,0
1,93.55,87.26,2,2,1977,-7.23,43.60,14.06,16.98,16.67,0.44,0
2,273.64,187.29,1,1,1996,-8.40,42.25,12.27,29.44,18.96,0.47,10
3,171.82,54.89,4,2,1996,-6.81,43.15,11.61,53.31,1.96,0.35,3
4,125.68,2153.49,1,1,1990,-8.76,42.92,10.04,70.20,4.72,0.09,12
...,...,...,...,...,...,...,...,...,...,...,...,...
19995,122.77,2285.95,6,2,2003,-8.34,43.63,5.74,65.57,12.38,0.69,0
19996,62.59,33.16,2,2,1970,-6.77,42.62,14.61,21.46,21.57,0.15,0
19997,144.47,427.37,4,1,1988,-6.87,42.00,14.74,16.98,0.72,0.33,6
19998,139.52,0.00,6,2,2014,-7.21,43.38,18.15,15.45,14.61,0.28,0


## Linear regression

In [33]:
# Separación de datos en train y test
X_train, X_test, y_train, y_test = train_test_split(
    X, target, test_size=0.2, random_state=42  # 20% para test
)

# Entrenamiento del modelo de regresión lineal 
model = LinearRegression()
model.fit(X_train, y_train)

# Predicción de los valores de test
y_pred_int = model.predict(X_test)

mae = mean_absolute_error(y_test, y_pred_int)
print("MAE:", mae)

resultados = pd.DataFrame({
    'real': y_test,
    'prediccion': y_pred_int
})

# Guardar en un archivo CSV
#resultados.to_csv('trains.csv',index=False)

MAE: 43835.58043423437


In [34]:
# Regresión linear eliminando id y poniendo a 0 los NaN

df_test = pd.read_csv('./data/test.csv', delimiter=',')
df_test=df3.copy()
for elm in df_test:
    if(isinstance(df_test[elm].iloc[1],str)):
        df_test=df_test.drop(elm,axis=1)
df_test_final=df_test.fillna(0)
if "id" in df_test_final.columns:
    X_final = df_test_final.drop(columns=["id"])
else:
    X_final = df_test_final.copy()
y_pred = model.predict(X_final)

mae = mean_absolute_error(y_test, y_pred_int)
print("MAE:", mae)

resultados = pd.DataFrame({
    'id': df_test["id"],
    'prezo_euros': y_pred
})

#resultados.to_csv('predicciones2.csv',index=False)

ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- prezo_euros


In [ ]:
# Regresión linear eliminando id y poniendo a mediana los NaN

df_test = pd.read_csv('./data/test.csv', delimiter=',')
for elm in df_test:
    if(isinstance(df_test[elm].iloc[1],str)):
        df_test=df_test.drop(elm,axis=1)
df_test_final=df_test.fillna(df_test.median(numeric_only=True))
X_final = df_test_final.drop(columns=["id"])

y_pred = model.predict(X_final)
resultados = pd.DataFrame({
    'id': df_test["id"],
    'prezo_euros': y_pred
})

resultados.to_csv('predicciones3.csv',index=False)

## Random forest

#### Base

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, target, test_size=0.2, random_state=42  # 20% para test
)

model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)
y_pred_int = model.predict(X_test)


mae = mean_absolute_error(y_test, y_pred_int)
print("MAE:", mae)

resultados = pd.DataFrame({
    'real': y_test,
    'prediccion': y_pred_int
})

# Guardar en un archivo CSV
resultados.to_csv('trains.csv',index=False)

MAE: 39774.65418


In [ ]:
df_test = pd.read_csv('./data/test.csv', delimiter=',')
for elm in df_test:
    if(isinstance(df_test[elm].iloc[1],str)):
        #print(elm)
        df_test=df_test.drop(elm,axis=1)
df_test_final=df_test.fillna(df_test.median(numeric_only=True))
X_final = df_test_final.drop(columns=["id"])

y_pred = model.predict(X_final)
resultados = pd.DataFrame({
    'id': df_test["id"],
    'prezo_euros': y_pred
})

resultados.to_csv('predicciones4.csv',index=False)

##### Selección de columnas

In [ ]:
# Aplicar SelectKBest para seleccionar características relevantes
ncomponentes=4 #se cambia aqui la cantidad de componentes a la que se puede reducir
selector = SelectKBest(score_func=f_classif, k=ncomponentes)
dfreducido=selector.fit_transform(X,target)

# Obtener las puntuaciones de importancia
feature_scores = selector.scores_
feature_names = X.columns

# Crear un DataFrame con las puntuaciones y ordenarlo
scores_df = pd.DataFrame({'Feature': feature_names, 'Score': feature_scores})
scores_df = scores_df.sort_values(by="Score", ascending=False)

# Mostrar las características seleccionadas
selected_features = feature_names[selector.get_support()]
print("\n\nCARACTERÍSTICAS SELECCIONADAS\n\n", selected_features)

# Mostrar las puntuaciones de las características
print("\n\nSCORES")
print(scores_df)



CARACTERÍSTICAS SELECCIONADAS

 Index(['superficie_interior_m2', 'superficie_exterior_m2',
       'distancia_centro_km', 'numero_arboles_xardin'],
      dtype='object')


SCORES
                               Feature      Score
1               superficie_exterior_m2  36.091595
11               numero_arboles_xardin   8.981673
0               superficie_interior_m2   5.334339
8                  distancia_centro_km   5.173231
10                indice_criminalidade   1.277655
2                   numero_habitacions   1.084005
6                             latitude   1.054078
4                     ano_construccion   1.028849
9                  distancia_escola_km   1.018464
3                         numero_banos   1.017178
7   temperatura_media_mes_construccion   0.996617
5                            lonxitude   0.994775


In [ ]:
X_reducido = X[selected_features]
X_train, X_test, y_train, y_test = train_test_split(
    X_reducido, target, test_size=0.2, random_state=42  # 20% para test
)

model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)
y_pred_int = model.predict(X_test)


mae = mean_absolute_error(y_test, y_pred_int)
print("MAE:", mae)

MAE: 44854.24890593308


El error aumenta cuando se reducen las columnas así que descartaremos esta opción por ahora

#### Incluir columnas string

(Antes se eliminaban todas las columnas que no fueran numéricas)

In [ ]:
X_preprocesado = df1.copy()

# 1. Eliminar la columna 'id'
X_preprocesado = X_preprocesado.drop(columns=['id'])

# 2. Separar la variable objetivo
y = X_preprocesado['prezo_euros']
X_preprocesado = X_preprocesado.drop(columns=['prezo_euros'])

# 3. Codificar columnas de texto

# Label Encoding para 'eficiencia_enerxetica' (porque tiene orden)
le = LabelEncoder()
X_preprocesado['eficiencia_enerxetica'] = le.fit_transform(X_preprocesado['eficiencia_enerxetica'])

# One-Hot Encoding para las otras columnas de texto
X_preprocesado = pd.get_dummies(X_preprocesado, columns=[
    'tipo_edificacion',
    'calidade_materiais',
    'cor_favorita_propietario',
    'acceso_transporte_publico',
    'orientacion'
])

# 4. Rellenar NaN en las columnas numéricas
X_preprocesado = X_preprocesado.fillna(X_preprocesado.median(numeric_only=True))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X_preprocesado, y, test_size=0.2, random_state=42  # 20% para test
)

#model = RandomForestRegressor(n_estimators=100, random_state=42); #BASE
model = RandomForestRegressor(n_estimators=400, random_state=42, max_depth=30, min_samples_split=5, min_samples_leaf=4)
#'n_estimators': [100, 200, 300],  # Número de árboles
#    'max_depth': [None, 10, 20, 30],  # Profundidad de los árboles
#    'min_samples_split': [2, 5, 10],  # Mínimo número de muestras para dividir un nodo
#    'min_samples_leaf': [1, 2, 4],    # Mínimo número de muestras en un nodo hoja
#    'max_features': ['auto', 'sqrt', 'log2'],  # Número de características a considerar para la división
#    'bootstrap': [True, False]  # Si se usa bootstrap (muestreo con reemplazo)
#model = DecisionTreeRegressor(random_state=42);
#model = KNeighborsRegressor(n_neighbors=15)

model.fit(X_train, y_train)
y_pred_int = model.predict(X_test)


mae = mean_absolute_error(y_test, y_pred_int)
print("MAE:", mae)

MAE: 32304.397951442097


In [ ]:
#Entrenamos con todo el dataset de train
#model = RandomForestRegressor(n_estimators=100, random_state=42)
model = RandomForestRegressor(n_estimators=400, random_state=42, max_depth=30, min_samples_split=5, min_samples_leaf=4)
model.fit(X_preprocesado, y)

RandomForestRegressor(max_depth=30, min_samples_leaf=4, min_samples_split=5,
                      n_estimators=400, random_state=42)

In [ ]:
df_test = pd.read_csv('./data/test.csv', delimiter=',')

X_preprocesado = df_test.copy()

# 1. Eliminar la columna 'id'
X_preprocesado = X_preprocesado.drop(columns=['id'])

# 3. Codificar columnas de texto

# Label Encoding para 'eficiencia_enerxetica' (porque tiene orden)
le = LabelEncoder()
X_preprocesado['eficiencia_enerxetica'] = le.fit_transform(X_preprocesado['eficiencia_enerxetica'])

# One-Hot Encoding para las otras columnas de texto
X_preprocesado = pd.get_dummies(X_preprocesado, columns=[
    'tipo_edificacion',
    'calidade_materiais',
    'cor_favorita_propietario',
    'acceso_transporte_publico',
    'orientacion'
])

# 4. Rellenar NaN en las columnas numéricas
X_preprocesado = X_preprocesado.fillna(X_preprocesado.median(numeric_only=True))

y_pred = model.predict(X_preprocesado)
resultados = pd.DataFrame({
    'id': df_test["id"],
    'prezo_euros': y_pred
})

#resultados.to_csv('predicciones10.csv',index=False)
resultados.to_csv('predicciones12.csv',index=False)

## Filtrado

In [ ]:
def filtrar_outliers(df, columnas):
    df_filtr = df1.copy()
    for col in columnas:
        Q1 = df[col].quantile(0.25) #Cuartiles
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1 #Rango intercuartilico
        lim_inf = Q1 - 2 * IQR #Definimos los límites
        lim_sup = Q3 + 2 * IQR
        
        n_antes = df_filtr.shape[0]
        df_filtr = df_filtr[(df_filtr[col] >= lim_inf) & (df_filtr[col] <= lim_sup)] #Filtramos los datos que estén fuera de los limites
        n_despues = df_filtr.shape[0]
        print(f"{col}: eliminadas {n_antes - n_despues} filas")
    return df_filtr

In [ ]:
X_preprocesado = df1.copy()

# 1. Eliminar la columna 'id'
X_preprocesado = X_preprocesado.drop(columns=['id'])

# 3. Codificar columnas de texto

# Label Encoding para 'eficiencia_enerxetica' (porque tiene orden)
le = LabelEncoder()
X_preprocesado['eficiencia_enerxetica'] = le.fit_transform(X_preprocesado['eficiencia_enerxetica'])

# One-Hot Encoding para las otras columnas de texto
X_preprocesado = pd.get_dummies(X_preprocesado, columns=[
    'tipo_edificacion',
    'calidade_materiais',
    'cor_favorita_propietario',
    'acceso_transporte_publico',
    'orientacion'
])

columnas_outliers = [
    'superficie_interior_m2',
    'superficie_exterior_m2',
    'numero_habitacions',
    'numero_banos',
    'ano_construccion',
    'lonxitude',
    'latitude',
    'temperatura_media_mes_construccion',
    'distancia_centro_km',
    'distancia_escola_km',
    'indice_criminalidade',
    'numero_arboles_xardin',
]

X_preprocesado = filtrar_outliers(X_preprocesado, columnas_outliers)

# 4. Rellenar NaN en las columnas numéricas
X_preprocesado = X_preprocesado.fillna(X_preprocesado.median(numeric_only=True))

y = X_preprocesado['prezo_euros']
X_preprocesado = X_preprocesado.drop(columns=['prezo_euros'])

superficie_interior_m2: eliminadas 32 filas
superficie_exterior_m2: eliminadas 3132 filas
numero_habitacions: eliminadas 0 filas
numero_banos: eliminadas 0 filas
ano_construccion: eliminadas 0 filas
lonxitude: eliminadas 0 filas
latitude: eliminadas 0 filas
temperatura_media_mes_construccion: eliminadas 0 filas
distancia_centro_km: eliminadas 4 filas
distancia_escola_km: eliminadas 16 filas
indice_criminalidade: eliminadas 3 filas
numero_arboles_xardin: eliminadas 0 filas


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X_preprocesado, y, test_size=0.2, random_state=42  # 20% para test
)

model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)
y_pred_int = model.predict(X_test)


mae = mean_absolute_error(y_test, y_pred_int)
print("MAE:", mae)

ValueError: could not convert string to float: 'Chalet adosado'

In [ ]:
df_test = pd.read_csv('./data/test.csv', delimiter=',')

X_preprocesado = df_test.copy()

# 1. Eliminar la columna 'id'
X_preprocesado = X_preprocesado.drop(columns=['id'])

# 3. Codificar columnas de texto

# Label Encoding para 'eficiencia_enerxetica' (porque tiene orden)
le = LabelEncoder()
X_preprocesado['eficiencia_enerxetica'] = le.fit_transform(X_preprocesado['eficiencia_enerxetica'])

# One-Hot Encoding para las otras columnas de texto
X_preprocesado = pd.get_dummies(X_preprocesado, columns=[
    'tipo_edificacion',
    'calidade_materiais',
    'cor_favorita_propietario',
    'acceso_transporte_publico',
    'orientacion'
])

# 4. Rellenar NaN en las columnas numéricas
X_preprocesado = X_preprocesado.fillna(X_preprocesado.median(numeric_only=True))

y_pred = model.predict(X_preprocesado)
resultados = pd.DataFrame({
    'id': df_test["id"],
    'prezo_euros': y_pred
})

resultados.to_csv('predicciones6.csv',index=False)

AttributeError: 'RandomForestRegressor' object has no attribute 'estimators_'

In [ ]:
def normalizar(train,test):
  escalar = MinMaxScaler()
  normalizado_train = escalar.fit_transform(train)
  normalizado_train = pd.DataFrame(normalizado_train, columns=train.columns)
  normalizado_test = escalar.transform(test)
  normalizado_test = pd.DataFrame(normalizado_test, columns=test.columns)
  return normalizado_train,normalizado_test

In [ ]:
X_train_norm, X_test_norm = normalizar(X_train, X_test)

In [ ]:
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train_norm, y_train)
y_pred = model.predict(X_test_norm)


mae = mean_absolute_error(y_test, y_pred)
print("MAE:", mae)

MAE: 32503.974922499998


In [ ]:
df_test = pd.read_csv('./data/test.csv', delimiter=',')

X_preprocesado = df_test.copy()

# 1. Eliminar la columna 'id'
X_preprocesado = X_preprocesado.drop(columns=['id'])

# 3. Codificar columnas de texto

# Label Encoding para 'eficiencia_enerxetica' (porque tiene orden)
le = LabelEncoder()
X_preprocesado['eficiencia_enerxetica'] = le.fit_transform(X_preprocesado['eficiencia_enerxetica'])

# One-Hot Encoding para las otras columnas de texto
X_preprocesado = pd.get_dummies(X_preprocesado, columns=[
    'tipo_edificacion',
    'calidade_materiais',
    'cor_favorita_propietario',
    'acceso_transporte_publico',
    'orientacion'
])

# 4. Rellenar NaN en las columnas numéricas
X_preprocesado = X_preprocesado.fillna(X_preprocesado.median(numeric_only=True))

_, X_preprocesado = normalizar(X_train, X_preprocesado)

y_pred = model.predict(X_preprocesado)
resultados = pd.DataFrame({
    'id': df_test["id"],
    'prezo_euros': y_pred
})

resultados.to_csv('predicciones7.csv',index=False)

ValueError: could not convert string to float: 'Chalet adosado'

In [ ]:
X_preprocesado = df1.copy()

# 1. Eliminar la columna 'id'
X_preprocesado = X_preprocesado.drop(columns=['id'])

# 3. Codificar columnas de texto

# Label Encoding para 'eficiencia_enerxetica' (porque tiene orden)
le = LabelEncoder()
X_preprocesado['eficiencia_enerxetica'] = le.fit_transform(X_preprocesado['eficiencia_enerxetica'])

# One-Hot Encoding para las otras columnas de texto
X_preprocesado = pd.get_dummies(X_preprocesado, columns=[
    'tipo_edificacion',
    'calidade_materiais',
    'cor_favorita_propietario',
    'acceso_transporte_publico',
    'orientacion'
])

# 4. Rellenar NaN en las columnas numéricas
X_preprocesado = X_preprocesado.fillna(X_preprocesado.median(numeric_only=True))

y = X_preprocesado['prezo_euros']
X_preprocesado = X_preprocesado.drop(columns=['prezo_euros'])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X_preprocesado, y, test_size=0.2, random_state=42  # 20% para test
)

X_train_norm, X_test_norm = normalizar(X_train, X_test)

model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)
y_pred_int = model.predict(X_test)


mae = mean_absolute_error(y_test, y_pred_int)
print("MAE:", mae)

MAE: 32512.2101775


In [ ]:
df_test = pd.read_csv('./data/test.csv', delimiter=',')

X_preprocesado = df_test.copy()

# 1. Eliminar la columna 'id'
X_preprocesado = X_preprocesado.drop(columns=['id'])

# 3. Codificar columnas de texto

# Label Encoding para 'eficiencia_enerxetica' (porque tiene orden)
le = LabelEncoder()
X_preprocesado['eficiencia_enerxetica'] = le.fit_transform(X_preprocesado['eficiencia_enerxetica'])

# One-Hot Encoding para las otras columnas de texto
X_preprocesado = pd.get_dummies(X_preprocesado, columns=[
    'tipo_edificacion',
    'calidade_materiais',
    'cor_favorita_propietario',
    'acceso_transporte_publico',
    'orientacion'
])

# 4. Rellenar NaN en las columnas numéricas
X_preprocesado = X_preprocesado.fillna(X_preprocesado.median(numeric_only=True))

_, X_preprocesado = normalizar(X_train, X_preprocesado)

y_pred = model.predict(X_preprocesado)
resultados = pd.DataFrame({
    'id': df_test["id"],
    'prezo_euros': y_pred
})

resultados.to_csv('predicciones8.csv',index=False)

## Modelos 

### Linear Regression

### Random Forest

### KNN

In [2]:
#  1. Cargar dataset de entrenamiento 
df = pd.read_csv('./data/train.csv', delimiter=',')

#  2. Guardar y separar target 
target = df['prezo_euros']
X = df.drop(columns=['id', 'prezo_euros'])

#  3. Codificar variables categóricas 
# Label Encoding para eficiencia_enerxetica
le = LabelEncoder()
X['eficiencia_enerxetica'] = le.fit_transform(X['eficiencia_enerxetica'])

# One-Hot Encoding para el resto
X = pd.get_dummies(X, columns=[
    'tipo_edificacion',
    'calidade_materiais',
    'cor_favorita_propietario',
    'acceso_transporte_publico',
    'orientacion'
])

#  4. Rellenar NaNs con mediana 
X = X.fillna(X.median(numeric_only=True))

#  5. Dividir en train/test 
X_train, X_test, y_train, y_test = train_test_split(X, target, test_size=0.2, random_state=42)

#  6. Normalizar 
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

#  7. Entrenar modelo KNN 
knn = KNeighborsRegressor(n_neighbors=5)
knn.fit(X_train_scaled, y_train)

#  8. Evaluar 
y_pred_test = knn.predict(X_test_scaled)
mae = mean_absolute_error(y_test, y_pred_test)
print("MAE del modelo KNN:", mae)

#  9. Cargar test.csv 
df_test = pd.read_csv('./data/test.csv', delimiter=',')

# Guardar el ID
id_test = df_test['id']
df_test = df_test.drop(columns=['id'])

#  10. Codificar igual que en entrenamiento 
df_test['eficiencia_enerxetica'] = le.transform(df_test['eficiencia_enerxetica'])

df_test = pd.get_dummies(df_test, columns=[
    'tipo_edificacion',
    'calidade_materiais',
    'cor_favorita_propietario',
    'acceso_transporte_publico',
    'orientacion'
])

# Alinear columnas con X_train
df_test = df_test.reindex(columns=X_train.columns, fill_value=0)

# Rellenar NaNs
df_test = df_test.fillna(X_train.median(numeric_only=True))

#  11. Normalizar 
X_test_final = scaler.transform(df_test)

#  12. Predecir 
y_pred_final = knn.predict(X_test_final)

#  13. Guardar resultados 
resultados = pd.DataFrame({
    'id': id_test,
    'prezo_euros': y_pred_final
})
resultados.to_csv('predicciones11.csv', index=False)


c:\Users\lucia\AppData\Local\Programs\Python\Python39\lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
found 0 physical cores < 1
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "c:\Users\lucia\AppData\Local\Programs\Python\Python39\lib\site-packages\joblib\externals\loky\backend\context.py", line 282, in _count_physical_cores
    raise ValueError(f"found {cpu_count_physical} physical cores < 1")


MAE del modelo KNN: 55564.73325


#### MLP

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Modelo MLP
mlp = MLPRegressor(
    hidden_layer_sizes=(256, 128, 64, 32, 16),
    activation='relu',
    solver='adam',
    alpha=0.01,
    learning_rate_init=0.002,
    max_iter=3000,
    early_stopping=True,
    n_iter_no_change=20,
    tol=1e-4,
    validation_fraction=0.25,
    random_state=42
)

mlp.fit(X_train_scaled, y_train)
y_pred = mlp.predict(X_test_scaled)

# Evaluación
mae = mean_absolute_error(y_test, y_pred)
print("MAE con MLPRegressor:", mae)


MAE con MLPRegressor: 32677.795686290756


In [ ]:
# Cargar test.csv
df_test = pd.read_csv('./data/test.csv', delimiter=',')

# Guardar id
id_test = df_test['id']

# Eliminar columna id
X_test = df_test.drop(columns=['id'])

# Codificar 'eficiencia_enerxetica' con el encoder ya entrenado
X_test['eficiencia_enerxetica'] = le.transform(X_test['eficiencia_enerxetica'])

# One-Hot Encoding (igual que en entrenamiento)
X_test = pd.get_dummies(X_test, columns=[
    'tipo_edificacion',
    'calidade_materiais',
    'cor_favorita_propietario',
    'acceso_transporte_publico',
    'orientacion'
])

# Alinear columnas con las del entrenamiento
X_test = X_test.reindex(columns=X_train.columns, fill_value=0)

# Rellenar NaN con la mediana de entrenamiento
X_test = X_test.fillna(X_train.median(numeric_only=True))

# Normalizar usando el scaler entrenado
X_test_scaled = scaler.transform(X_test)

# Predecir
y_pred = mlp.predict(X_test_scaled)

# Guardar resultados
resultados = pd.DataFrame({
    'id': id_test,
    'prezo_euros': y_pred
})
resultados.to_csv('predicciones13.csv', index=False)

## Primer intento de Ensemble

In [ ]:
#IMPORTANTE, BAGGING Y STACKING TIENEN FUNCIONES QUE LAS HACEN SOLAS, SIN EMBARGO BLENDING NO

In [ ]:
#Stacking
#Se hace (opcional) cross validation del modelo sobre el dataset
#Se hace el entrenamiento con el train completo
    #Se repite para todos los modelos que queramos meter a la stack
#Se sacan sus predicciones
#Se entrena el METAMODELO con las predicciones (Y SOLO LAS PREDICCIONES)

In [ ]:
#Blending
#Se separa el dframe de train en una partición de train real y una holdout.
#Se entrenan x modelos con el train real
#Se pide la predicción de los resultados de cada uno al holdout
#Se entrena un modelo final (metamodelo) con tanto el train de holdout como las predicciones de el resto de modelos para holdout
#Se saca las predicciones del metamodelo al test final

## STACKING

In [ ]:
# RESULTADOS PARCIALES

#metamodelo=LinearRegression() PARECE SER LO MEJOR POR LO QUE SE USA EN LAS TRES

#1: 30876.624 En 3 minutos
'''
modelos_iniciales= [
    ('KNN',KNeighborsRegressor(n_neighbors=5)),
    ('RForest',RandomForestRegressor(n_estimators=100, random_state=42)),
    ('MLP',MLPRegressor(hidden_layer_sizes=(256, 128, 64, 32, 16),activation='relu',solver='adam',alpha=0.01,learning_rate_init=0.002,max_iter=3000,early_stopping=True,n_iter_no_change=20,tol=1e-4,validation_fraction=0.25,random_state=42))
]
'''
#2: 30011.960 En 3 minutos
'''
modelos_iniciales= [
    ('KNN',KNeighborsRegressor(n_neighbors=5)),
    ('RForest',RandomForestRegressor(n_estimators=100, random_state=42)),
    ('MLP',MLPRegressor(hidden_layer_sizes=(256, 128, 64, 32, 16),activation='relu',solver='adam',alpha=0.01,learning_rate_init=0.002,max_iter=3000,early_stopping=True,n_iter_no_change=20,tol=1e-4,validation_fraction=0.25,random_state=42)),
    ('Ridge', Ridge(alpha=1.0)),
    ('HGB', HistGradientBoostingRegressor(max_iter=200, random_state=42))
]'''
#3: 30009.615 En 4.50 minutos
'''
modelos_iniciales = [
    ('KNN', KNeighborsRegressor(n_neighbors=5)),
    ('RForest', RandomForestRegressor(n_estimators=100, random_state=42)),
    ('MLP',MLPRegressor(hidden_layer_sizes=(256, 128, 64, 32, 16),activation='relu',solver='adam',alpha=0.01,learning_rate_init=0.002,max_iter=3000,early_stopping=True,n_iter_no_change=20,tol=1e-4,validation_fraction=0.25,random_state=42)),
    ('Ridge', Ridge(alpha=1.0)),
    ('HGB', HistGradientBoostingRegressor(max_iter=200, random_state=42)),
    ('SVR', SVR(kernel='rbf', C=10)),
]
'''


In [ ]:
#Preprocesado (sacado del KNN)

#  1. Cargar dataset de entrenamiento 
df = pd.read_csv('./data/train.csv', delimiter=',')

#  2. Guardar y separar target 
target = df['prezo_euros']
X = df.drop(columns=['id', 'prezo_euros'])

#  3. Codificar variables categóricas 
# Label Encoding para eficiencia_enerxetica
le = LabelEncoder()
X['eficiencia_enerxetica'] = le.fit_transform(X['eficiencia_enerxetica'])

# One-Hot Encoding para el resto
X = pd.get_dummies(X, columns=[
    'tipo_edificacion',
    'calidade_materiais',
    'cor_favorita_propietario',
    'acceso_transporte_publico',
    'orientacion'
])

#  4. Rellenar NaNs con mediana 
X = X.fillna(X.median(numeric_only=True))

#  5. Dividir en train/test temporal #NO SE USA EN ESTA VERSION
X_temp, X_test, y_temp, y_test = train_test_split(X, target, test_size=0.2, random_state=42)

# 5.2 Se hace un split extra, aquí se sacan el X e Y del train reales y un X e y para el metamodelo
X_train, X_meta_train_final, y_train, y_meta = train_test_split(X_temp, y_temp, test_size=0.2, random_state=42)

#  6. Normalizar 
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

#  6. Normalizar el metamodelo
X_meta_scaled = scaler.transform(X_meta_train_final)

########################################################################################
########################################################################################

# NUEVO: se usa el stacked model
# Los estimators son los modelos iniciales que se usan. El final estimator es el metamodelo

#OPCION A, 30011.96057308429

modelos_iniciales= [
    ('KNN',KNeighborsRegressor(n_neighbors=5)),
    ('RForest',RandomForestRegressor(n_estimators=100, random_state=42)),
    ('MLP',MLPRegressor(hidden_layer_sizes=(256, 128, 64, 32, 16),activation='relu',solver='adam',alpha=0.01,learning_rate_init=0.002,max_iter=3000,early_stopping=True,n_iter_no_change=20,tol=1e-4,validation_fraction=0.25,random_state=42)),
    ('Ridge', Ridge(alpha=1.0)),
    ('HGB', HistGradientBoostingRegressor(max_iter=200, random_state=42))
]
#OPCION B, 30009.615173543672 #UN MINUTO MAS LENTO
'''
modelos_iniciales = [
    ('KNN', KNeighborsRegressor(n_neighbors=5)),
    ('RForest', RandomForestRegressor(n_estimators=100, random_state=42)),
    ('MLP',MLPRegressor(hidden_layer_sizes=(256, 128, 64, 32, 16),activation='relu',solver='adam',alpha=0.01,learning_rate_init=0.002,max_iter=3000,early_stopping=True,n_iter_no_change=20,tol=1e-4,validation_fraction=0.25,random_state=42)),
    ('Ridge', Ridge(alpha=1.0)),
    ('HGB', HistGradientBoostingRegressor(max_iter=200, random_state=42)),
    ('SVR', SVR(kernel='rbf', C=10)),
]
'''

metamodelo=LinearRegression()

modelo_completo = StackingRegressor(
    estimators=modelos_iniciales,
    final_estimator=metamodelo,
    cv=5,passthrough=False) #el cv es cross validation y passthrough pasa los X además de las predicciones

modelo_completo.fit(X_train_scaled, y_train)

y_pred = modelo_completo.predict(X_test_scaled)
mae = mean_absolute_error(y_test, y_pred)
print("MAE (Stacking Ensemble):", mae)





#  9. Cargar test.csv 
df_test = pd.read_csv('./data/test.csv', delimiter=',')

# Guardar el ID
id_test = df_test['id']
df_test = df_test.drop(columns=['id'])

#  10. Codificar igual que en entrenamiento 
df_test['eficiencia_enerxetica'] = le.transform(df_test['eficiencia_enerxetica'])

df_test = pd.get_dummies(df_test, columns=[
    'tipo_edificacion',
    'calidade_materiais',
    'cor_favorita_propietario',
    'acceso_transporte_publico',
    'orientacion'
])

# Alinear columnas con X_train
df_test = df_test.reindex(columns=X_train.columns, fill_value=0)

# Rellenar NaNs
df_test = df_test.fillna(X_train.median(numeric_only=True))

#  11. Normalizar 
X_test_final = scaler.transform(df_test)

#  12. Predecir 
y_pred_final = modelo_completo.predict(X_test_final)

#  13. Guardar resultados 
resultados = pd.DataFrame({
    'id': id_test,
    'prezo_euros': y_pred_final
})
resultados.to_csv('predicciones_stacking2.csv', index=False)

MAE (Stacking Ensemble): 30009.615173543672


## BLENDING

In [ ]:
#Preprocesado (sacado del KNN)

#  1. Cargar dataset de entrenamiento 
df = pd.read_csv('./data/train.csv', delimiter=',')

#  2. Guardar y separar target 
target = df['prezo_euros']
X = df.drop(columns=['id', 'prezo_euros'])

#  3. Codificar variables categóricas 
# Label Encoding para eficiencia_enerxetica
le = LabelEncoder()
X['eficiencia_enerxetica'] = le.fit_transform(X['eficiencia_enerxetica'])

# One-Hot Encoding para el resto
X = pd.get_dummies(X, columns=[
    'tipo_edificacion',
    'calidade_materiais',
    'cor_favorita_propietario',
    'acceso_transporte_publico',
    'orientacion'
])

#  4. Rellenar NaNs con mediana 
X = X.fillna(X.median(numeric_only=True))

#X_train es para entrenar a los modelos iniciales
#X_temp se pasa al segundo split
#X_meta es de donde sacamos las predicciones para el metamodelo
#X_test es X_test
X_train, X_temp, y_train, y_temp = train_test_split(X, target, test_size=0.4, random_state=42)
X_meta_train_final, X_test, y_meta, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

#  6. Normalizar 
#  6.1 Primero setear el scaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

#  6.2 Luego aplicar sobre el modelo y el metamodelo
X_meta_scaled = scaler.transform(X_meta_train_final)
X_test_scaled = scaler.transform(X_test)

########################################################################################
########################################################################################

#Indicamos cuales van a ser los modelos iniciales a entrenar
modelos_iniciales= [
    KNeighborsRegressor(n_neighbors=5),
    RandomForestRegressor(n_estimators=100, random_state=42),
    MLPRegressor(hidden_layer_sizes=(256, 128, 64, 32, 16),activation='relu',solver='adam',alpha=0.01,learning_rate_init=0.002,max_iter=3000,early_stopping=True,n_iter_no_change=20,tol=1e-4,validation_fraction=0.25,random_state=42),
    Ridge(alpha=1.0),
    HistGradientBoostingRegressor(max_iter=200, random_state=42)
]

#Indicamos cual va a ser el metamodelo a usar
#con este y las iniciales sobre 29950
#metamodelo=LinearRegression()
metamodelo= MLPRegressor(hidden_layer_sizes=(64, 32), activation='relu',solver='adam', max_iter=2000, early_stopping=True,random_state=42)
#SVR por probar


predicciones_iniciales=[]
predicciones_test=[]

#Por cada uno de los modelos, se entrena y se pasa sus predicciones
for modelo in tqdm(modelos_iniciales, desc="Entrenando modelos..."):
    nombremodel=modelo.__class__.__name__
    tqdm.write(f"Entrenando: {nombremodel}")
    #PRIMERO ENTRENAMOS
    modelo.fit(X_train_scaled,y_train)
    #PRIMERO PREDECIMOS SOBRE UN SET PARA TENER PREDICCIONES
    predicciones_iniciales.append(modelo.predict(X_meta_scaled))
    #AHORA PREDECIMOS SOBRE EL TEST
    predicciones_test.append(modelo.predict(X_test_scaled))

X_metamodelo=np.column_stack(predicciones_iniciales)
X_metamodelo_final=np.column_stack(predicciones_test)
########################################################################################
########################################################################################



metamodelo.fit(X_metamodelo,y_meta)

y_pred = metamodelo.predict(X_metamodelo_final)

mae = mean_absolute_error(y_test, y_pred)
print("MAE (Blending Ensemble):", mae)



## TRATAR DE REPETIR EL PROCESO CON EL REAL

#  9. Cargar test.csv 
df_test = pd.read_csv('./data/test.csv', delimiter=',')

# Guardar el ID
id_test = df_test['id']
df_test = df_test.drop(columns=['id'])

#  10. Codificar igual que en entrenamiento 
df_test['eficiencia_enerxetica'] = le.transform(df_test['eficiencia_enerxetica'])

df_test = pd.get_dummies(df_test, columns=[
    'tipo_edificacion',
    'calidade_materiais',
    'cor_favorita_propietario',
    'acceso_transporte_publico',
    'orientacion'
])

# Alinear columnas con X_train
df_test = df_test.reindex(columns=X_train.columns, fill_value=0)

# Rellenar NaNs
df_test = df_test.fillna(X_train.median(numeric_only=True))

#  11. Normalizar 
X_test_final = scaler.transform(df_test)

predicciones_reales=[]
for modelo in modelos_iniciales:
    predicciones_reales.append(modelo.predict(X_test_final))

stack_predicciones=np.column_stack(predicciones_reales)

#  12. Predecir 
y_pred_final = metamodelo.predict(stack_predicciones)

#  13. Guardar resultados 
resultados = pd.DataFrame({
    'id': id_test,
    'prezo_euros': y_pred_final
})
print('PREDICCIONES GENERADAS')
resultados.to_csv('predicciones_stacking4.csv', index=False)

Entrenando modelos...:  20%|██        | 1/5 [00:00<00:00,  9.90it/s]

Entrenando: KNeighborsRegressor
Entrenando: RandomForestRegressor


Entrenando modelos...:  40%|████      | 2/5 [00:09<00:17,  5.77s/it]

Entrenando: MLPRegressor


Entrenando modelos...:  60%|██████    | 3/5 [00:28<00:23, 11.82s/it]

Entrenando: Ridge
Entrenando: HistGradientBoostingRegressor


Entrenando modelos...: 100%|██████████| 5/5 [00:29<00:00,  5.85s/it]


MAE (Blending Ensemble): 29498.945546361527
PREDICCIONES GENERADAS


In [ ]:
#Preprocesado (sacado del KNN)

#  1. Cargar dataset de entrenamiento 
df = pd.read_csv('./data/train.csv', delimiter=',')

#  2. Guardar y separar target 
target = df['prezo_euros']
X = df.drop(columns=['id', 'prezo_euros'])

#  3. Codificar variables categóricas 
# Label Encoding para eficiencia_enerxetica
le = LabelEncoder()
X['eficiencia_enerxetica'] = le.fit_transform(X['eficiencia_enerxetica'])

# One-Hot Encoding para el resto
X = pd.get_dummies(X, columns=[
    'tipo_edificacion',
    'calidade_materiais',
    'cor_favorita_propietario',
    'acceso_transporte_publico',
    'orientacion'
])

#  4. Rellenar NaNs con mediana 
X = X.fillna(X.median(numeric_only=True))

#X_train es para entrenar a los modelos iniciales
#X_temp se pasa al segundo split
#X_meta es de donde sacamos las predicciones para el metamodelo
#X_test es X_test
X_train_full, X_test, y_train_full, y_test = train_test_split(X, target, test_size=0.2, random_state=42)

#  6. Normalizar 
#  6.1 Primero setear el scaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_full)
X_test_scaled = scaler.transform(X_test)

########################################################################################
########################################################################################

#Indicamos cuales van a ser los modelos iniciales a entrenar
modelos_iniciales= [
    KNeighborsRegressor(n_neighbors=5),
    RandomForestRegressor(n_estimators=100, random_state=42),
    MLPRegressor(hidden_layer_sizes=(256, 128, 64, 32, 16),activation='relu',solver='adam',alpha=0.01,learning_rate_init=0.002,max_iter=3000,early_stopping=True,n_iter_no_change=20,tol=1e-4,validation_fraction=0.25,random_state=42),
    Ridge(alpha=1.0),
    HistGradientBoostingRegressor(max_iter=200, random_state=42),
    #HASTA AQUI 29536.1468

    CatBoostRegressor(
        iterations=1000,
        learning_rate=0.05,
        depth=6,
        loss_function='MAE',
        random_seed=42,
        verbose=0,
        od_type='Iter',
        od_wait=30),
    LGBMRegressor(
        n_estimators=1000,
        learning_rate=0.05,
        max_depth=6,
        num_leaves=31,
        min_child_samples=20,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42
    ),
    XGBRegressor(
        n_estimators=1000,
        learning_rate=0.05,
        max_depth=6,
        subsample=0.8,
        colsample_bytree=0.8,
        reg_alpha=0.1,  
        reg_lambda=1.0,
        objective='reg:squarederror',
        random_state=42
    )
]
'''
    XGBRegressor(n_estimators=200, learning_rate=0.1, max_depth=6, random_state=42),
    LGBMRegressor(n_estimators=200, learning_rate=0.05, random_state=42),
    CatBoostRegressor(iterations=200, learning_rate=0.05, depth=6, verbose=0, random_state=42)'''

#Indicamos cual va a ser el metamodelo a usar
#con este y las iniciales sobre 29950
#metamodelo=LinearRegression()
metamodelo= MLPRegressor(hidden_layer_sizes=(64, 32), activation='relu',solver='adam', max_iter=2000, early_stopping=True,random_state=42)
#SVR por probar

X_meta_train = np.zeros((len(X_train_scaled), len(modelos_iniciales)))
X_meta_test = np.zeros((len(X_test_scaled), len(modelos_iniciales)))

splits=5
kf = KFold(n_splits=splits, shuffle=True, random_state=42)
modelos_entrenados_finales = []  # Para guardar modelos entrenados finales

for modelo_idx, modelo in enumerate(tqdm(modelos_iniciales, desc="Entrenando modelos...")):
    nombremodel = modelo.__class__.__name__
    tqdm.write(f"Entrenando: {nombremodel}")
    
    predicciones_ext = [None] * len(X_train_scaled)
    predicciones_int = []

    for train_idx, val_idx in kf.split(X_train_scaled):
        X_train_int, y_train_int = X_train_scaled[train_idx], y_train_full.iloc[train_idx]
        X_val = X_train_scaled[val_idx]

        modelo_cv = clone(modelo)
        modelo_cv.fit(X_train_int, y_train_int)

        preds_val = modelo_cv.predict(X_val)
        for idx, pred in zip(val_idx, preds_val):
            predicciones_ext[idx] = pred

        predicciones_int.append(modelo_cv.predict(X_test_scaled))

    for i, pred in enumerate(predicciones_ext):
        X_meta_train[i, modelo_idx] = pred

    avg_test_preds = np.mean(predicciones_int, axis=0)
    for i, pred in enumerate(avg_test_preds):
        X_meta_test[i, modelo_idx] = pred

    modelo_entrenado = clone(modelo)
    modelo_entrenado.fit(X_train_scaled, y_train_full)
    modelos_entrenados_finales.append(modelo_entrenado)


metamodelo.fit(X_meta_train, y_train_full)
y_pred_provisional = metamodelo.predict(X_meta_test)

mae_metamodelo = mean_absolute_error(y_test, y_pred_provisional)
print(f"\n\nMAE metamodelo: {mae_metamodelo:.4f}")


#  9. Cargar test.csv 
df_test = pd.read_csv('./data/test.csv', delimiter=',')

# Guardar el ID
id_test = df_test['id']
df_test = df_test.drop(columns=['id'])

#  10. Codificar igual que en entrenamiento 
df_test['eficiencia_enerxetica'] = le.transform(df_test['eficiencia_enerxetica'])

df_test = pd.get_dummies(df_test, columns=[
    'tipo_edificacion',
    'calidade_materiais',
    'cor_favorita_propietario',
    'acceso_transporte_publico',
    'orientacion'
])

# Alinear columnas con X_train
df_test = df_test.reindex(columns=X_train_full.columns, fill_value=0)

# Rellenar NaNs
df_test = df_test.fillna(X_train_full.median(numeric_only=True))

#  11. Normalizar 
X_test_final = scaler.transform(df_test)

predicciones_reales=[]
for modelo in modelos_entrenados_finales:
    predicciones_reales.append(modelo.predict(X_test_final))

stack_predicciones=np.column_stack(predicciones_reales)

#  12. Predecir 
y_pred_final = metamodelo.predict(stack_predicciones)

#  13. Guardar resultados 
resultados = pd.DataFrame({
    'id': id_test,
    'prezo_euros': y_pred_final
})
print('PREDICCIONES GENERADAS')
resultados.to_csv('predicciones_stacking5.csv', index=False)

Entrenando modelos...:   0%|          | 0/8 [00:00<?, ?it/s]

Entrenando: KNeighborsRegressor


Entrenando modelos...:  12%|█▎        | 1/8 [00:00<00:03,  2.15it/s]

Entrenando: RandomForestRegressor


Entrenando modelos...:  25%|██▌       | 2/8 [01:42<05:59, 59.99s/it]

...

Entrenando modelos...:  88%|████████▊ | 7/8 [05:03<00:30, 30.16s/it]

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

Entrenando modelos...: 100%|██████████| 8/8 [05:10<00:00, 38.76s/it]




MAE metamodelo: 28889.4232
PREDICCIONES GENERADAS


c:\Users\Anxo\Documents\Python\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


In [ ]:
### ESTA CELDA ES UN COPIA Y PEGA DE LA DE ARRIBA PERO DA RESULTADOS DISTINTOS
# WTF??


#Preprocesado (sacado del KNN)

#  1. Cargar dataset de entrenamiento 
df = pd.read_csv('./data/train.csv', delimiter=',')

#  2. Guardar y separar target 
target = df['prezo_euros']
X = df.drop(columns=['id', 'prezo_euros'])

#  3. Codificar variables categóricas 
# Label Encoding para eficiencia_enerxetica
le = LabelEncoder()
X['eficiencia_enerxetica'] = le.fit_transform(X['eficiencia_enerxetica'])

# One-Hot Encoding para el resto
X = pd.get_dummies(X, columns=[
    'tipo_edificacion',
    'calidade_materiais',
    'cor_favorita_propietario',
    'acceso_transporte_publico',
    'orientacion'
])

#  4. Rellenar NaNs con mediana 
X = X.fillna(X.median(numeric_only=True))

#X_train es para entrenar a los modelos iniciales
#X_temp se pasa al segundo split
#X_meta es de donde sacamos las predicciones para el metamodelo
#X_test es X_test
X_train_full, X_test, y_train_full, y_test = train_test_split(X, target, test_size=0.2, random_state=42)

#  6. Normalizar 
#  6.1 Primero setear el scaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_full)
X_test_scaled = scaler.transform(X_test)

########################################################################################
########################################################################################

#Indicamos cuales van a ser los modelos iniciales a entrenar
modelos_iniciales= [
    KNeighborsRegressor(n_neighbors=5),
    RandomForestRegressor(n_estimators=100, random_state=42),
    MLPRegressor(hidden_layer_sizes=(256, 128, 64, 32, 16),activation='relu',solver='adam',alpha=0.01,learning_rate_init=0.002,max_iter=3000,early_stopping=True,n_iter_no_change=20,tol=1e-4,validation_fraction=0.25,random_state=42),
    Ridge(alpha=1.0),
    HistGradientBoostingRegressor(max_iter=200, random_state=42),
    #HASTA AQUI 29536.1468

    CatBoostRegressor(
        iterations=1000,
        learning_rate=0.05,
        depth=6,
        loss_function='MAE',
        random_seed=42,
        verbose=0,
        od_type='Iter',
        od_wait=30),
    LGBMRegressor(
        n_estimators=1000,
        learning_rate=0.05,
        max_depth=6,
        num_leaves=31,
        min_child_samples=20,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42
    ),
    XGBRegressor(
        n_estimators=1000,
        learning_rate=0.05,
        max_depth=6,
        subsample=0.8,
        colsample_bytree=0.8,
        reg_alpha=0.1,  
        reg_lambda=1.0,
        objective='reg:squarederror',
        random_state=42
    )
]
'''
    XGBRegressor(n_estimators=200, learning_rate=0.1, max_depth=6, random_state=42),
    LGBMRegressor(n_estimators=200, learning_rate=0.05, random_state=42),
    CatBoostRegressor(iterations=200, learning_rate=0.05, depth=6, verbose=0, random_state=42)'''

#Indicamos cual va a ser el metamodelo a usar
#con este y las iniciales sobre 29950
#metamodelo=LinearRegression()
metamodelo= MLPRegressor(hidden_layer_sizes=(64, 32), activation='relu',solver='adam', max_iter=2000, early_stopping=True,random_state=42)
#SVR por probar

X_meta_train = np.zeros((len(X_train_scaled), len(modelos_iniciales)))
X_meta_test = np.zeros((len(X_test_scaled), len(modelos_iniciales)))

splits=5
kf = KFold(n_splits=splits, shuffle=True, random_state=42)
modelos_entrenados_finales = []  # Para guardar modelos entrenados finales

for modelo_idx, modelo in enumerate(tqdm(modelos_iniciales, desc="Entrenando modelos...")):
    nombremodel = modelo.__class__.__name__
    tqdm.write(f"Entrenando: {nombremodel}")
    
    predicciones_ext = [None] * len(X_train_scaled)
    predicciones_int = []

    for train_idx, val_idx in kf.split(X_train_scaled):
        X_train_int, y_train_int = X_train_scaled[train_idx], y_train_full.iloc[train_idx]
        X_val = X_train_scaled[val_idx]

        modelo_cv = clone(modelo)
        modelo_cv.fit(X_train_int, y_train_int)

        preds_val = modelo_cv.predict(X_val)
        for idx, pred in zip(val_idx, preds_val):
            predicciones_ext[idx] = pred

        predicciones_int.append(modelo_cv.predict(X_test_scaled))

    for i, pred in enumerate(predicciones_ext):
        X_meta_train[i, modelo_idx] = pred

    avg_test_preds = np.mean(predicciones_int, axis=0)
    for i, pred in enumerate(avg_test_preds):
        X_meta_test[i, modelo_idx] = pred

    modelo_entrenado = clone(modelo)
    modelo_entrenado.fit(X_train_scaled, y_train_full)
    modelos_entrenados_finales.append(modelo_entrenado)


metamodelo.fit(X_meta_train, y_train_full)
y_pred_provisional = metamodelo.predict(X_meta_test)

mae_metamodelo = mean_absolute_error(y_test, y_pred_provisional)
print(f"\n\nMAE metamodelo: {mae_metamodelo:.4f}")


#  9. Cargar test.csv 
df_test = pd.read_csv('./data/test.csv', delimiter=',')

# Guardar el ID
id_test = df_test['id']
df_test = df_test.drop(columns=['id'])

#  10. Codificar igual que en entrenamiento 
df_test['eficiencia_enerxetica'] = le.transform(df_test['eficiencia_enerxetica'])

df_test = pd.get_dummies(df_test, columns=[
    'tipo_edificacion',
    'calidade_materiais',
    'cor_favorita_propietario',
    'acceso_transporte_publico',
    'orientacion'
])

# Alinear columnas con X_train
df_test = df_test.reindex(columns=X_train_full.columns, fill_value=0)

# Rellenar NaNs
df_test = df_test.fillna(X_train_full.median(numeric_only=True))

#  11. Normalizar 
X_test_final = scaler.transform(df_test)

predicciones_reales=[]
for modelo in modelos_entrenados_finales:
    predicciones_reales.append(modelo.predict(X_test_final))

stack_predicciones=np.column_stack(predicciones_reales)

#  12. Predecir 
y_pred_final = metamodelo.predict(stack_predicciones)

#  13. Guardar resultados 
resultados = pd.DataFrame({
    'id': id_test,
    'prezo_euros': y_pred_final
})
print('PREDICCIONES GENERADAS')
resultados.to_csv('predicciones_stacking5.csv', index=False)

Entrenando modelos...:   0%|          | 0/8 [00:00<?, ?it/s]

Entrenando: KNeighborsRegressor


Entrenando modelos...:  12%|█▎        | 1/8 [00:01<00:08,  1.19s/it]

Entrenando: RandomForestRegressor


Entrenando modelos...:  25%|██▌       | 2/8 [01:28<05:09, 51.66s/it]

...

Entrenando modelos...:  88%|████████▊ | 7/8 [05:16<00:34, 34.91s/it]

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Entrenando: XGBRegressor


Entrenando modelos...: 100%|██████████| 8/8 [05:29<00:00, 41.22s/it]




MAE metamodelo: 29049.4860
PREDICCIONES GENERADAS


c:\Users\lucia\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


In [ ]:
#### ENTRENANDO CON EL DATASET DE TRAIN COMPLETO ###

#Preprocesado (sacado del KNN)

#  1. Cargar dataset de entrenamiento 
df = pd.read_csv('./data/train.csv', delimiter=',')

#  2. Guardar y separar target 
target = df['prezo_euros']
X = df.drop(columns=['id', 'prezo_euros'])

#  3. Codificar variables categóricas 
# Label Encoding para eficiencia_enerxetica
le = LabelEncoder()
X['eficiencia_enerxetica'] = le.fit_transform(X['eficiencia_enerxetica'])

# One-Hot Encoding para el resto
X = pd.get_dummies(X, columns=[
    'tipo_edificacion',
    'calidade_materiais',
    'cor_favorita_propietario',
    'acceso_transporte_publico',
    'orientacion'
])

#  4. Rellenar NaNs con mediana 
X = X.fillna(X.median(numeric_only=True))

#X_train es para entrenar a los modelos iniciales
#X_temp se pasa al segundo split
#X_meta es de donde sacamos las predicciones para el metamodelo
#X_test es X_test
#X_train_full, X_test, y_train_full, y_test = train_test_split(X, target, test_size=0.2, random_state=42)
X_train_full = X.copy()
y_train_full = target.copy()

#  6. Normalizar 
#  6.1 Primero setear el scaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_full)
X_test_scaled = scaler.transform(X_train_full)
#X_test_scaled = scaler.transform(X_test)

########################################################################################
########################################################################################

#Indicamos cuales van a ser los modelos iniciales a entrenar
modelos_iniciales= [
    KNeighborsRegressor(n_neighbors=5),
    RandomForestRegressor(n_estimators=100, random_state=42),
    MLPRegressor(hidden_layer_sizes=(256, 128, 64, 32, 16),activation='relu',solver='adam',alpha=0.01,learning_rate_init=0.002,max_iter=3000,early_stopping=True,n_iter_no_change=20,tol=1e-4,validation_fraction=0.25,random_state=42),
    Ridge(alpha=1.0),
    HistGradientBoostingRegressor(max_iter=200, random_state=42),
    #HASTA AQUI 29536.1468

    CatBoostRegressor(
        iterations=1000,
        learning_rate=0.05,
        depth=6,
        loss_function='MAE',
        random_seed=42,
        verbose=0,
        od_type='Iter',
        od_wait=30),
    LGBMRegressor(
        n_estimators=1000,
        learning_rate=0.05,
        max_depth=6,
        num_leaves=31,
        min_child_samples=20,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42
    ),
    XGBRegressor(
        n_estimators=1000,
        learning_rate=0.05,
        max_depth=6,
        subsample=0.8,
        colsample_bytree=0.8,
        reg_alpha=0.1,  
        reg_lambda=1.0,
        objective='reg:squarederror',
        random_state=42
    )
]
'''
    XGBRegressor(n_estimators=200, learning_rate=0.1, max_depth=6, random_state=42),
    LGBMRegressor(n_estimators=200, learning_rate=0.05, random_state=42),
    CatBoostRegressor(iterations=200, learning_rate=0.05, depth=6, verbose=0, random_state=42)'''

#Indicamos cual va a ser el metamodelo a usar
#con este y las iniciales sobre 29950
#metamodelo=LinearRegression()
metamodelo= MLPRegressor(hidden_layer_sizes=(64, 32), activation='relu',solver='adam', max_iter=2000, early_stopping=True,random_state=42)
#SVR por probar

X_meta_train = np.zeros((len(X_train_scaled), len(modelos_iniciales)))
X_meta_test = np.zeros((len(X_test_scaled), len(modelos_iniciales)))

splits=5
kf = KFold(n_splits=splits, shuffle=True, random_state=42)
modelos_entrenados_finales = []  # Para guardar modelos entrenados finales

for modelo_idx, modelo in enumerate(tqdm(modelos_iniciales, desc="Entrenando modelos...")):
    nombremodel = modelo.__class__.__name__
    tqdm.write(f"Entrenando: {nombremodel}")
    
    predicciones_ext = [None] * len(X_train_scaled)
    predicciones_int = []

    for train_idx, val_idx in kf.split(X_train_scaled):
        X_train_int, y_train_int = X_train_scaled[train_idx], y_train_full.iloc[train_idx]
        X_val = X_train_scaled[val_idx]

        modelo_cv = clone(modelo)
        modelo_cv.fit(X_train_int, y_train_int)

        preds_val = modelo_cv.predict(X_val)
        for idx, pred in zip(val_idx, preds_val):
            predicciones_ext[idx] = pred

        predicciones_int.append(modelo_cv.predict(X_test_scaled))

    for i, pred in enumerate(predicciones_ext):
        X_meta_train[i, modelo_idx] = pred

    avg_test_preds = np.mean(predicciones_int, axis=0)
    for i, pred in enumerate(avg_test_preds):
        X_meta_test[i, modelo_idx] = pred

    modelo_entrenado = clone(modelo)
    modelo_entrenado.fit(X_train_scaled, y_train_full)
    modelos_entrenados_finales.append(modelo_entrenado)


metamodelo.fit(X_meta_train, y_train_full)
y_pred_provisional = metamodelo.predict(X_meta_test)

#mae_metamodelo = mean_absolute_error(y_test, y_pred_provisional)
#print(f"\n\nMAE metamodelo: {mae_metamodelo:.4f}")


#  9. Cargar test.csv 
df_test = pd.read_csv('./data/test.csv', delimiter=',')

# Guardar el ID
id_test = df_test['id']
df_test = df_test.drop(columns=['id'])

#  10. Codificar igual que en entrenamiento 
df_test['eficiencia_enerxetica'] = le.transform(df_test['eficiencia_enerxetica'])

df_test = pd.get_dummies(df_test, columns=[
    'tipo_edificacion',
    'calidade_materiais',
    'cor_favorita_propietario',
    'acceso_transporte_publico',
    'orientacion'
])

# Alinear columnas con X_train
df_test = df_test.reindex(columns=X_train_full.columns, fill_value=0)

# Rellenar NaNs
df_test = df_test.fillna(X_train_full.median(numeric_only=True))

#  11. Normalizar 
X_test_final = scaler.transform(df_test)

predicciones_reales=[]
for modelo in modelos_entrenados_finales:
    predicciones_reales.append(modelo.predict(X_test_final))

stack_predicciones=np.column_stack(predicciones_reales)

#  12. Predecir 
y_pred_final = metamodelo.predict(stack_predicciones)

#  13. Guardar resultados 
resultados = pd.DataFrame({
    'id': id_test,
    'prezo_euros': y_pred_final
})
print('PREDICCIONES GENERADAS')
resultados.to_csv('predicciones_stacking7.csv', index=False)


Entrenando modelos...:   0%|          | 0/8 [00:00<?, ?it/s]

Entrenando: KNeighborsRegressor


c:\Users\lucia\AppData\Local\Programs\Python\Python313\Lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
found 0 physical cores < 1
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "c:\Users\lucia\AppData\Local\Programs\Python\Python313\Lib\site-packages\joblib\externals\loky\backend\context.py", line 282, in _count_physical_cores
    raise ValueError(f"found {cpu_count_physical} physical cores < 1")
Entrenando modelos...:  12%|█▎        | 1/8 [00:02<00:20,  2.98s/it]

Entrenando: RandomForestRegressor


Entrenando modelos...:  25%|██▌       | 2/8 [02:47<09:49, 98.17s/it]

...

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001922 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2032
[LightGBM] [Info] Number of data points in the train set: 20000, number of used features: 32
[LightGBM] [Info] Start training from score 223402.814500
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best 

Entrenando modelos...:  88%|████████▊ | 7/8 [06:18<00:34, 34.86s/it]

Entrenando: XGBRegressor


Entrenando modelos...: 100%|██████████| 8/8 [06:34<00:00, 49.35s/it]
c:\Users\lucia\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


PREDICCIONES GENERADAS


In [ ]:
### OPTIMIZAR HIPRARAMETROS DE XGBRegressor

param_grid = {
    'n_estimators': [300, 500, 800],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [4, 6, 8],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'reg_alpha': [0.0, 0.1, 1.0],
    'reg_lambda': [0.5, 1.0, 2.0]
}

model_xgb = XGBRegressor(objective='reg:squarederror', random_state=42)
search_xgb = RandomizedSearchCV(model_xgb, param_distributions=param_grid, 
                                 n_iter=15, scoring='neg_mean_absolute_error',
                                 cv=3, verbose=1, random_state=42, n_jobs=-1)

search_xgb.fit(X_train_scaled, y_train_full)
best_xgb = search_xgb.best_estimator_
print("Mejor XGB:", search_xgb.best_params_)


Fitting 3 folds for each of 15 candidates, totalling 45 fits
Mejor XGB: {'subsample': 1.0, 'reg_lambda': 1.0, 'reg_alpha': 1.0, 'n_estimators': 300, 'max_depth': 4, 'learning_rate': 0.1, 'colsample_bytree': 0.8}


In [ ]:
# OPTIMIZAR HIPERPARAMETROS DE LGBMRegressor

param_grid = {
    'iterations': [500, 800, 1000],
    'learning_rate': [0.01, 0.05, 0.1],
    'depth': [4, 6, 8],
    'l2_leaf_reg': [1, 3, 5, 7]
}

model_cat = CatBoostRegressor(loss_function='MAE', verbose=0, random_seed=42)
search_cat = RandomizedSearchCV(model_cat, param_distributions=param_grid,
                                n_iter=15, scoring='neg_mean_absolute_error',
                                cv=3, verbose=1, random_state=42, n_jobs=-1)

search_cat.fit(X_train_scaled, y_train_full)
best_cat = search_cat.best_estimator_
print("Mejor CatBoost:", search_cat.best_params_)


Fitting 3 folds for each of 15 candidates, totalling 45 fits
Mejor CatBoost: {'learning_rate': 0.05, 'l2_leaf_reg': 5, 'iterations': 1000, 'depth': 6}


In [ ]:
# OPTIMIZAR HIPERPARAMETROS DE LGBMRegressor

param_grid = {
    'n_estimators': [300, 500, 800],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [4, 6, 8, -1],
    'num_leaves': [20, 31, 50],
    'min_child_samples': [10, 20, 30],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0]
}

model_lgbm = LGBMRegressor(random_state=42)
search_lgbm = RandomizedSearchCV(
    model_lgbm,
    param_distributions=param_grid,
    n_iter=15,
    scoring='neg_mean_absolute_error',
    cv=3,
    verbose=1,
    random_state=42,
    n_jobs=-1
)

search_lgbm.fit(X_train_scaled, y_train_full)
best_lgbm = search_lgbm.best_estimator_
print("Mejor LGBM:", search_lgbm.best_params_)


Fitting 3 folds for each of 15 candidates, totalling 45 fits
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.004600 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2032
[LightGBM] [Info] Number of data points in the train set: 20000, number of used features: 32
[LightGBM] [Info] Start training from score 223402.814500
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gai

In [ ]:
# OPTIMIZAR HIPERPARAMETROS DE KNeighborsRegressor

# Definir espacio de parámetros para KNeighborsRegressor
param_grid_knn = {
    'n_neighbors': np.arange(1, 20),  # Número de vecinos
    'weights': ['uniform', 'distance'],  # Cómo se ponderan los vecinos
    'metric': ['euclidean', 'manhattan', 'minkowski'],  # Métrica de distancia
}

knn_model = KNeighborsRegressor()

# Realizar búsqueda aleatoria
search_knn = RandomizedSearchCV(
    knn_model,
    param_distributions=param_grid_knn,
    n_iter=10,  # Número de combinaciones aleatorias
    scoring='neg_mean_absolute_error',  # Evaluar con MAE negativo
    cv=3,  # Validación cruzada
    verbose=1,
    random_state=42,
    n_jobs=-1
)

search_knn.fit(X_train_scaled, y_train_full)
best_knn = search_knn.best_estimator_
print("Mejor KNN:", search_knn.best_params_)


Fitting 3 folds for each of 10 candidates, totalling 30 fits
Mejor KNN: {'weights': 'distance', 'n_neighbors': np.int64(16), 'metric': 'manhattan'}


In [ ]:
# OPTIMIZAR HIPERPARAMETROS DE RandomForestRegressor

# Definir espacio de parámetros para RandomForestRegressor
param_grid_rf = {
    'n_estimators': [100, 200, 300, 400],  # Número de árboles
    'max_depth': [None, 10, 20, 30],  # Profundidad máxima de los árboles
    'min_samples_split': [2, 5, 10],  # Mínimo número de muestras para dividir un nodo
    'min_samples_leaf': [1, 2, 4],  # Mínimo número de muestras en una hoja
    'bootstrap': [True, False],  # Usar bootstrap
}

rf_model = RandomForestRegressor(random_state=42)

# Realizar búsqueda aleatoria
search_rf = RandomizedSearchCV(
    rf_model,
    param_distributions=param_grid_rf,
    n_iter=10,  # Número de combinaciones aleatorias
    scoring='neg_mean_absolute_error',  # Evaluar con MAE negativo
    cv=3,  # Validación cruzada
    verbose=1,
    random_state=42,
    n_jobs=-1
)

search_rf.fit(X_train_scaled, y_train_full)
best_rf = search_rf.best_estimator_
print("Mejor RandomForest:", search_rf.best_params_)


Fitting 3 folds for each of 10 candidates, totalling 30 fits


KeyboardInterrupt: 

In [ ]:
# OPTIMIZAR HIPERPARAMETROS DE HistGradientBoostingRegressor

# Definir espacio de parámetros para HistGradientBoostingRegressor
param_grid_hgb = {
    'max_iter': [100, 200, 300],  # Número de iteraciones
    'learning_rate': [0.01, 0.05, 0.1],  # Tasa de aprendizaje
    'max_depth': [3, 5, 10, None],  # Profundidad máxima de los árboles
    'min_samples_leaf': [20, 50, 100],  # Mínimo número de muestras en una hoja
    'l2_regularization': [0, 0.1, 0.5],  # Regularización L2
}

hgb_model = HistGradientBoostingRegressor(random_state=42)

# Realizar búsqueda aleatoria
search_hgb = RandomizedSearchCV(
    hgb_model,
    param_distributions=param_grid_hgb,
    n_iter=10,  # Número de combinaciones aleatorias
    scoring='neg_mean_absolute_error',  # Evaluar con MAE negativo
    cv=3,  # Validación cruzada
    verbose=1,
    random_state=42,
    n_jobs=-1
)

search_hgb.fit(X_train_scaled, y_train_full)
best_hgb = search_hgb.best_estimator_
print("Mejor HistGradientBoosting:", search_hgb.best_params_)


Fitting 3 folds for each of 10 candidates, totalling 30 fits
Mejor HistGradientBoosting: {'min_samples_leaf': 50, 'max_iter': 300, 'max_depth': 5, 'learning_rate': 0.1, 'l2_regularization': 0.1}


In [ ]:
### PRUEBA CON LAS OPTIMIZACIONES DE TODOOS LOS MODELOS
#MAE metamodelo: 28784.8810

#BESTMODELOS
#Mejor KNN: {'weights': 'distance', 'n_neighbors': np.int64(16), 'metric': 'manhattan'}
#Mejor HistGradientBoosting: {'min_samples_leaf': 50, 'max_iter': 300, 'max_depth': 5, 'learning_rate': 0.1, 'l2_regularization': 0.1}
#Mejor RandomForest: {'n_estimators': 200, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_depth': 20, 'bootstrap': True}
#Mejor LGBM: {'subsample': 0.8, 'num_leaves': 50, 'n_estimators': 300, 'min_child_samples': 30, 'max_depth': 6, 'learning_rate': 0.05, 'colsample_bytree': 0.6}
#Mejor CatBoost: {'learning_rate': 0.05, 'l2_leaf_reg': 5, 'iterations': 1000, 'depth': 6}
#Mejor XGB: {'subsample': 1.0, 'reg_lambda': 1.0, 'reg_alpha': 1.0, 'n_estimators': 300, 'max_depth': 4, 'learning_rate': 0.1, 'colsample_bytree': 0.8}


#Preprocesado (sacado del KNN)

#  1. Cargar dataset de entrenamiento 
df = pd.read_csv('./data/train.csv', delimiter=',')

#  2. Guardar y separar target 
target = df['prezo_euros']
X = df.drop(columns=['id', 'prezo_euros'])

#  3. Codificar variables categóricas 
# Label Encoding para eficiencia_enerxetica
le = LabelEncoder()
X['eficiencia_enerxetica'] = le.fit_transform(X['eficiencia_enerxetica'])

# One-Hot Encoding para el resto
X = pd.get_dummies(X, columns=[
    'tipo_edificacion',
    'calidade_materiais',
    'cor_favorita_propietario',
    'acceso_transporte_publico',
    'orientacion'
])

#  4. Rellenar NaNs con mediana 
X = X.fillna(X.median(numeric_only=True))

#X_train es para entrenar a los modelos iniciales
#X_temp se pasa al segundo split
#X_meta es de donde sacamos las predicciones para el metamodelo
#X_test es X_test
X_train_full, X_test, y_train_full, y_test = train_test_split(X, target, test_size=0.2, random_state=42)

#  6. Normalizar 
#  6.1 Primero setear el scaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_full)
X_test_scaled = scaler.transform(X_test)

########################################################################################
########################################################################################

#Indicamos cuales van a ser los modelos iniciales a entrenar
modelos_iniciales = [
    best_knn,
    best_rf,
    best_hgb,
    best_cat,
    best_lgbm,
    best_xgb   
]
'''
    XGBRegressor(n_estimators=200, learning_rate=0.1, max_depth=6, random_state=42),
    LGBMRegressor(n_estimators=200, learning_rate=0.05, random_state=42),
    CatBoostRegressor(iterations=200, learning_rate=0.05, depth=6, verbose=0, random_state=42)'''

#Indicamos cual va a ser el metamodelo a usar
#con este y las iniciales sobre 29950
#metamodelo=LinearRegression()
metamodelo= MLPRegressor(hidden_layer_sizes=(64, 32), activation='relu',solver='adam', max_iter=2000, early_stopping=True,random_state=42)
#SVR por probar

X_meta_train = np.zeros((len(X_train_scaled), len(modelos_iniciales)))
X_meta_test = np.zeros((len(X_test_scaled), len(modelos_iniciales)))

splits=5
kf = KFold(n_splits=splits, shuffle=True, random_state=42)
modelos_entrenados_finales = []  # Para guardar modelos entrenados finales

for modelo_idx, modelo in enumerate(tqdm(modelos_iniciales, desc="Entrenando modelos...")):
    nombremodel = modelo.__class__.__name__
    tqdm.write(f"Entrenando: {nombremodel}")
    
    predicciones_ext = [None] * len(X_train_scaled)
    predicciones_int = []

    for train_idx, val_idx in kf.split(X_train_scaled):
        X_train_int, y_train_int = X_train_scaled[train_idx], y_train_full.iloc[train_idx]
        X_val = X_train_scaled[val_idx]

        modelo_cv = clone(modelo)
        modelo_cv.fit(X_train_int, y_train_int)

        preds_val = modelo_cv.predict(X_val)
        for idx, pred in zip(val_idx, preds_val):
            predicciones_ext[idx] = pred

        predicciones_int.append(modelo_cv.predict(X_test_scaled))

    for i, pred in enumerate(predicciones_ext):
        X_meta_train[i, modelo_idx] = pred

    avg_test_preds = np.mean(predicciones_int, axis=0)
    for i, pred in enumerate(avg_test_preds):
        X_meta_test[i, modelo_idx] = pred

    modelo_entrenado = clone(modelo)
    modelo_entrenado.fit(X_train_scaled, y_train_full)
    modelos_entrenados_finales.append(modelo_entrenado)


metamodelo.fit(X_meta_train, y_train_full)
y_pred_provisional = metamodelo.predict(X_meta_test)

mae_metamodelo = mean_absolute_error(y_test, y_pred_provisional)
print(f"\n\nMAE metamodelo: {mae_metamodelo:.4f}")


#  9. Cargar test.csv 
df_test = pd.read_csv('./data/test.csv', delimiter=',')

# Guardar el ID
id_test = df_test['id']
df_test = df_test.drop(columns=['id'])

#  10. Codificar igual que en entrenamiento 
df_test['eficiencia_enerxetica'] = le.transform(df_test['eficiencia_enerxetica'])

df_test = pd.get_dummies(df_test, columns=[
    'tipo_edificacion',
    'calidade_materiais',
    'cor_favorita_propietario',
    'acceso_transporte_publico',
    'orientacion'
])

# Alinear columnas con X_train
df_test = df_test.reindex(columns=X_train_full.columns, fill_value=0)

# Rellenar NaNs
df_test = df_test.fillna(X_train_full.median(numeric_only=True))

#  11. Normalizar 
X_test_final = scaler.transform(df_test)

predicciones_reales=[]
for modelo in modelos_entrenados_finales:
    predicciones_reales.append(modelo.predict(X_test_final))

stack_predicciones=np.column_stack(predicciones_reales)

#  12. Predecir 
y_pred_final = metamodelo.predict(stack_predicciones)

#  13. Guardar resultados 
resultados = pd.DataFrame({
    'id': id_test,
    'prezo_euros': y_pred_final
})
print('PREDICCIONES GENERADAS')
resultados.to_csv('predicciones_stacking5.csv', index=False) # Este csv no sirve para nada

Entrenando modelos...:   0%|          | 0/6 [00:00<?, ?it/s]

Entrenando: KNeighborsRegressor


Entrenando modelos...:  17%|█▋        | 1/6 [00:04<00:21,  4.38s/it]

Entrenando: RandomForestRegressor


Entrenando modelos...:  33%|███▎      | 2/6 [02:47<06:29, 97.47s/it]

...

Entrenando modelos...:  83%|████████▎ | 5/6 [03:34<00:32, 32.58s/it]

Entrenando: XGBRegressor


Entrenando modelos...: 100%|██████████| 6/6 [03:37<00:00, 36.25s/it]




MAE metamodelo: 28784.8810
PREDICCIONES GENERADAS


c:\Users\lucia\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


In [ ]:
#### ENTRENANDO CON EL DATASET DE TRAIN COMPLETO ###

#Preprocesado (sacado del KNN)

#  1. Cargar dataset de entrenamiento 
df = pd.read_csv('./data/train.csv', delimiter=',')

#  2. Guardar y separar target 
target = df['prezo_euros']
X = df.drop(columns=['id', 'prezo_euros'])

#  3. Codificar variables categóricas 
# Label Encoding para eficiencia_enerxetica
le = LabelEncoder()
X['eficiencia_enerxetica'] = le.fit_transform(X['eficiencia_enerxetica'])

# One-Hot Encoding para el resto
X = pd.get_dummies(X, columns=[
    'tipo_edificacion',
    'calidade_materiais',
    'cor_favorita_propietario',
    'acceso_transporte_publico',
    'orientacion'
])

#  4. Rellenar NaNs con mediana 
X = X.fillna(X.median(numeric_only=True))

#X_train es para entrenar a los modelos iniciales
#X_temp se pasa al segundo split
#X_meta es de donde sacamos las predicciones para el metamodelo
#X_test es X_test
#X_train_full, X_test, y_train_full, y_test = train_test_split(X, target, test_size=0.2, random_state=42)
X_train_full = X.copy()
y_train_full = target.copy()

#  6. Normalizar 
#  6.1 Primero setear el scaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_full)
X_test_scaled = scaler.transform(X_train_full)
#X_test_scaled = scaler.transform(X_test)

########################################################################################
########################################################################################

#Indicamos cuales van a ser los modelos iniciales a entrenar
modelos_iniciales = [
    best_knn,
    best_rf,
    best_hgb,
    best_cat, 
    best_lgbm, 
    best_xgb   
]
'''
    XGBRegressor(n_estimators=200, learning_rate=0.1, max_depth=6, random_state=42),
    LGBMRegressor(n_estimators=200, learning_rate=0.05, random_state=42),
    CatBoostRegressor(iterations=200, learning_rate=0.05, depth=6, verbose=0, random_state=42)'''

#Indicamos cual va a ser el metamodelo a usar
#con este y las iniciales sobre 29950
#metamodelo=LinearRegression()
metamodelo= MLPRegressor(hidden_layer_sizes=(64, 32), activation='relu',solver='adam', max_iter=2000, early_stopping=True,random_state=42)
#SVR por probar

X_meta_train = np.zeros((len(X_train_scaled), len(modelos_iniciales)))
X_meta_test = np.zeros((len(X_test_scaled), len(modelos_iniciales)))

splits=5
kf = KFold(n_splits=splits, shuffle=True, random_state=42)
modelos_entrenados_finales = []  # Para guardar modelos entrenados finales

for modelo_idx, modelo in enumerate(tqdm(modelos_iniciales, desc="Entrenando modelos...")):
    nombremodel = modelo.__class__.__name__
    tqdm.write(f"Entrenando: {nombremodel}")
    
    predicciones_ext = [None] * len(X_train_scaled)
    predicciones_int = []

    for train_idx, val_idx in kf.split(X_train_scaled):
        X_train_int, y_train_int = X_train_scaled[train_idx], y_train_full.iloc[train_idx]
        X_val = X_train_scaled[val_idx]

        modelo_cv = clone(modelo)
        modelo_cv.fit(X_train_int, y_train_int)

        preds_val = modelo_cv.predict(X_val)
        for idx, pred in zip(val_idx, preds_val):
            predicciones_ext[idx] = pred

        predicciones_int.append(modelo_cv.predict(X_test_scaled))

    for i, pred in enumerate(predicciones_ext):
        X_meta_train[i, modelo_idx] = pred

    avg_test_preds = np.mean(predicciones_int, axis=0)
    for i, pred in enumerate(avg_test_preds):
        X_meta_test[i, modelo_idx] = pred

    modelo_entrenado = clone(modelo)
    modelo_entrenado.fit(X_train_scaled, y_train_full)
    modelos_entrenados_finales.append(modelo_entrenado)


metamodelo.fit(X_meta_train, y_train_full)
y_pred_provisional = metamodelo.predict(X_meta_test)

#mae_metamodelo = mean_absolute_error(y_test, y_pred_provisional)
#print(f"\n\nMAE metamodelo: {mae_metamodelo:.4f}")

#  9. Cargar test.csv 
df_test = pd.read_csv('./data/test.csv', delimiter=',')

# Guardar el ID
id_test = df_test['id']
df_test = df_test.drop(columns=['id'])

#  10. Codificar igual que en entrenamiento 
df_test['eficiencia_enerxetica'] = le.transform(df_test['eficiencia_enerxetica'])

df_test = pd.get_dummies(df_test, columns=[
    'tipo_edificacion',
    'calidade_materiais',
    'cor_favorita_propietario',
    'acceso_transporte_publico',
    'orientacion'
])

# Alinear columnas con X_train
df_test = df_test.reindex(columns=X_train_full.columns, fill_value=0)

# Rellenar NaNs
df_test = df_test.fillna(X_train_full.median(numeric_only=True))

#  11. Normalizar 
X_test_final = scaler.transform(df_test)

predicciones_reales=[]
for modelo in modelos_entrenados_finales:
    predicciones_reales.append(modelo.predict(X_test_final))

stack_predicciones=np.column_stack(predicciones_reales)

#  12. Predecir 
y_pred_final = metamodelo.predict(stack_predicciones)

#  13. Guardar resultados 
resultados = pd.DataFrame({
    'id': id_test,
    'prezo_euros': y_pred_final
})
print('PREDICCIONES GENERADAS')
resultados.to_csv('predicciones_stacking8.csv', index=False)


Entrenando modelos...:   0%|          | 0/6 [00:00<?, ?it/s]

Entrenando: KNeighborsRegressor


Entrenando modelos...:  17%|█▋        | 1/6 [00:12<01:01, 12.26s/it]

Entrenando: RandomForestRegressor


Entrenando modelos...:  33%|███▎      | 2/6 [03:27<08:00, 120.12s/it]

...

Entrenando modelos...:  83%|████████▎ | 5/6 [04:12<00:36, 36.46s/it]

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

Entrenando modelos...: 100%|██████████| 6/6 [04:14<00:00, 42.36s/it]


PREDICCIONES GENERADAS


c:\Users\lucia\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


In [ ]:
#### ENTRENANDO CON EL DATASET DE TRAIN COMPLETO ###

#Preprocesado (sacado del KNN)

#  1. Cargar dataset de entrenamiento 
df = pd.read_csv('./data/train.csv', delimiter=',')

#  2. Guardar y separar target 
target = df['prezo_euros']
X = df.drop(columns=['id', 'prezo_euros'])

#  3. Codificar variables categóricas 
# Label Encoding para eficiencia_enerxetica
le = LabelEncoder()
X['eficiencia_enerxetica'] = le.fit_transform(X['eficiencia_enerxetica'])

# One-Hot Encoding para el resto
X = pd.get_dummies(X, columns=[
    'tipo_edificacion',
    'calidade_materiais',
    'cor_favorita_propietario',
    'acceso_transporte_publico',
    'orientacion'
])

#  4. Rellenar NaNs con mediana 
X = X.fillna(X.median(numeric_only=True))

#X_train es para entrenar a los modelos iniciales
#X_temp se pasa al segundo split
#X_meta es de donde sacamos las predicciones para el metamodelo
#X_test es X_test
#X_train_full, X_test, y_train_full, y_test = train_test_split(X, target, test_size=0.2, random_state=42)
X_train_full = X.copy()
y_train_full = target.copy()

#  6. Normalizar 
#  6.1 Primero setear el scaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_full)

In [ ]:
# OPTIMIZACIÓN MLP

param_grid_mlp = {
    'hidden_layer_sizes': [(64, 32), (128, 64), (256, 128, 64)],
    'activation': ['relu', 'tanh'],
    'solver': ['adam', 'sgd'],
    'max_iter': [1000, 2000],
    'learning_rate_init': [0.001, 0.01],
}

mlp = MLPRegressor(random_state=42)
search_mlp = RandomizedSearchCV(
    mlp,
    param_distributions=param_grid_mlp,
    n_iter=10,  # Número de combinaciones aleatorias
    scoring='neg_mean_absolute_error',  # Evaluar con MAE negativo
    cv=3,  # Validación cruzada
    verbose=1,
    random_state=42,
    n_jobs=-1
)

search_mlp.fit(X_train_scaled, y_train_full)

# Guardamos el modelo optimizado
best_mlp = search_mlp.best_estimator_
print("Mejores parámetros para MLP:", search_mlp.best_params_)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


c:\Users\Anxo\Documents\Python\lib\site-packages\sklearn\model_selection\_validation.py:528: FitFailedWarning: 
3 fits failed out of a total of 30.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
3 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\Anxo\Documents\Python\lib\site-packages\sklearn\model_selection\_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\Anxo\Documents\Python\lib\site-packages\sklearn\base.py", line 1389, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "c:\Users\Anxo\Documents\Python\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py", line 754, in fit
    return self._fit(X, y, i

Mejores parámetros para MLP: {'solver': 'adam', 'max_iter': 1000, 'learning_rate_init': 0.01, 'hidden_layer_sizes': (64, 32), 'activation': 'relu'}


## BESTMODELS

In [9]:
best_knn = KNeighborsRegressor(
    weights='distance',
    n_neighbors=int(np.int64(16)),
    metric='manhattan'
)
best_rf = RandomForestRegressor(
    n_estimators=200,
    min_samples_split=5,
    min_samples_leaf=1,
    max_depth=20,
    bootstrap=True,random_state=42
)
best_hgb = HistGradientBoostingRegressor(
    min_samples_leaf=50,
    max_iter=300,
    max_depth=5,
    learning_rate=0.1,
    l2_regularization=0.1,random_state=42
)
best_cat = CatBoostRegressor(
    learning_rate=0.05,
    l2_leaf_reg=5,
    iterations=1000,
    depth=6,
    verbose=0,random_state=42
)
best_lgbm = LGBMRegressor(
    subsample=0.8,
    num_leaves=50,
    n_estimators=300,
    min_child_samples=30,
    max_depth=6,
    learning_rate=0.05,
    colsample_bytree=0.6,random_state=42
)
best_xgb = XGBRegressor(
    subsample=1.0,
    reg_lambda=1.0,
    reg_alpha=1.0,
    n_estimators=300,
    max_depth=4,
    learning_rate=0.1,
    colsample_bytree=0.8,random_state=42
)
best_mlp = MLPRegressor(
    solver='adam',
    max_iter=1000,
    learning_rate_init=0.01,
    hidden_layer_sizes=(64, 32),
    activation='relu',random_state=42
)

In [ ]:
### PRUEBA CON LAS OPTIMIZACIONES DE TODOOS LOS MODELOS
## Y MLP OPTIMIZADO

#Preprocesado (sacado del KNN)

#  1. Cargar dataset de entrenamiento 
df = pd.read_csv('./data/train.csv', delimiter=',')

#  2. Guardar y separar target 
target = df['prezo_euros']
X = df.drop(columns=['id', 'prezo_euros'])

#  3. Codificar variables categóricas 
# Label Encoding para eficiencia_enerxetica
le = LabelEncoder()
X['eficiencia_enerxetica'] = le.fit_transform(X['eficiencia_enerxetica'])

# One-Hot Encoding para el resto
X = pd.get_dummies(X, columns=[
    'tipo_edificacion',
    'calidade_materiais',
    'cor_favorita_propietario',
    'acceso_transporte_publico',
    'orientacion'
])

#  4. Rellenar NaNs con mediana 
X = X.fillna(X.median(numeric_only=True))

#X_train es para entrenar a los modelos iniciales
#X_temp se pasa al segundo split
#X_meta es de donde sacamos las predicciones para el metamodelo
#X_test es X_test
X_train_full, X_test, y_train_full, y_test = train_test_split(X, target, test_size=0.2, random_state=42)

#  6. Normalizar 
#  6.1 Primero setear el scaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_full)
X_test_scaled = scaler.transform(X_test)

########################################################################################
########################################################################################

#Indicamos cuales van a ser los modelos iniciales a entrenar
modelos_iniciales = [
    best_knn,
    best_rf,
    best_hgb,
    best_cat,
    best_lgbm,
    best_xgb   
]
'''
    XGBRegressor(n_estimators=200, learning_rate=0.1, max_depth=6, random_state=42),
    LGBMRegressor(n_estimators=200, learning_rate=0.05, random_state=42),
    CatBoostRegressor(iterations=200, learning_rate=0.05, depth=6, verbose=0, random_state=42)'''

#Indicamos cual va a ser el metamodelo a usar
#con este y las iniciales sobre 29950
#metamodelo=LinearRegression()
#metamodelo= MLPRegressor(hidden_layer_sizes=(64, 32), activation='relu',solver='adam', max_iter=2000, early_stopping=True,random_state=42)
metamodelo= best_mlp
#SVR por probar

X_meta_train = np.zeros((len(X_train_scaled), len(modelos_iniciales)))
X_meta_test = np.zeros((len(X_test_scaled), len(modelos_iniciales)))

splits=5
kf = KFold(n_splits=splits, shuffle=True, random_state=42)
modelos_entrenados_finales = []  # Para guardar modelos entrenados finales

for modelo_idx, modelo in enumerate(tqdm(modelos_iniciales, desc="Entrenando modelos...")):
    nombremodel = modelo.__class__.__name__
    tqdm.write(f"Entrenando: {nombremodel}")
    
    predicciones_ext = [None] * len(X_train_scaled)
    predicciones_int = []

    for train_idx, val_idx in kf.split(X_train_scaled):
        X_train_int, y_train_int = X_train_scaled[train_idx], y_train_full.iloc[train_idx]
        X_val = X_train_scaled[val_idx]

        modelo_cv = clone(modelo)
        modelo_cv.fit(X_train_int, y_train_int)

        preds_val = modelo_cv.predict(X_val)
        for idx, pred in zip(val_idx, preds_val):
            predicciones_ext[idx] = pred

        predicciones_int.append(modelo_cv.predict(X_test_scaled))

    for i, pred in enumerate(predicciones_ext):
        X_meta_train[i, modelo_idx] = pred

    avg_test_preds = np.mean(predicciones_int, axis=0)
    for i, pred in enumerate(avg_test_preds):
        X_meta_test[i, modelo_idx] = pred

    modelo_entrenado = clone(modelo)
    modelo_entrenado.fit(X_train_scaled, y_train_full)
    modelos_entrenados_finales.append(modelo_entrenado)


metamodelo.fit(X_meta_train, y_train_full)
y_pred_provisional = metamodelo.predict(X_meta_test)

mae_metamodelo = mean_absolute_error(y_test, y_pred_provisional)
print(f"\n\nMAE metamodelo: {mae_metamodelo:.4f}")


#  9. Cargar test.csv 
df_test = pd.read_csv('./data/test.csv', delimiter=',')

# Guardar el ID
id_test = df_test['id']
df_test = df_test.drop(columns=['id'])

#  10. Codificar igual que en entrenamiento 
df_test['eficiencia_enerxetica'] = le.transform(df_test['eficiencia_enerxetica'])

df_test = pd.get_dummies(df_test, columns=[
    'tipo_edificacion',
    'calidade_materiais',
    'cor_favorita_propietario',
    'acceso_transporte_publico',
    'orientacion'
])

# Alinear columnas con X_train
df_test = df_test.reindex(columns=X_train_full.columns, fill_value=0)

# Rellenar NaNs
df_test = df_test.fillna(X_train_full.median(numeric_only=True))

#  11. Normalizar 
X_test_final = scaler.transform(df_test)

predicciones_reales=[]
for modelo in modelos_entrenados_finales:
    predicciones_reales.append(modelo.predict(X_test_final))

stack_predicciones=np.column_stack(predicciones_reales)

#  12. Predecir 
y_pred_final = metamodelo.predict(stack_predicciones)

#  13. Guardar resultados 
resultados = pd.DataFrame({
    'id': id_test,
    'prezo_euros': y_pred_final
})
print('PREDICCIONES GENERADAS')
##resultados.to_csv('predicciones_stacking5.csv', index=False) # Este csv no sirve para nada

Entrenando modelos...:   0%|          | 0/6 [00:00<?, ?it/s]

Entrenando: KNeighborsRegressor


Entrenando modelos...:  17%|█▋        | 1/6 [00:01<00:05,  1.13s/it]

Entrenando: RandomForestRegressor


Entrenando modelos...:  33%|███▎      | 2/6 [02:47<06:33, 98.48s/it]

...

Entrenando modelos...:  83%|████████▎ | 5/6 [03:10<00:26, 26.44s/it]

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

Entrenando modelos...: 100%|██████████| 6/6 [03:11<00:00, 31.95s/it]




MAE metamodelo: 28693.2851
PREDICCIONES GENERADAS


c:\Users\Anxo\Documents\Python\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


In [ ]:
#ESTE POR LO QUE SEA NO DA BIEN

#### ENTRENANDO CON EL DATASET DE TRAIN COMPLETO ###
## BEST MLP OPTIMIZADO

#Preprocesado (sacado del KNN)

#  1. Cargar dataset de entrenamiento 
df = pd.read_csv('./data/train.csv', delimiter=',')

#  2. Guardar y separar target 
target = df['prezo_euros']
X = df.drop(columns=['id', 'prezo_euros'])

#  3. Codificar variables categóricas 
# Label Encoding para eficiencia_enerxetica
le = LabelEncoder()
X['eficiencia_enerxetica'] = le.fit_transform(X['eficiencia_enerxetica'])

# One-Hot Encoding para el resto
X = pd.get_dummies(X, columns=[
    'tipo_edificacion',
    'calidade_materiais',
    'cor_favorita_propietario',
    'acceso_transporte_publico',
    'orientacion'
])

#  4. Rellenar NaNs con mediana 
X = X.fillna(X.median(numeric_only=True))

#X_train es para entrenar a los modelos iniciales
#X_temp se pasa al segundo split
#X_meta es de donde sacamos las predicciones para el metamodelo
#X_test es X_test
#X_train_full, X_test, y_train_full, y_test = train_test_split(X, target, test_size=0.2, random_state=42)
X_train_full = X.copy()
y_train_full = target.copy()

#  6. Normalizar 
#  6.1 Primero setear el scaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_full)
X_test_scaled = scaler.transform(X_train_full)
#X_test_scaled = scaler.transform(X_test)

########################################################################################
########################################################################################

#Indicamos cuales van a ser los modelos iniciales a entrenar
modelos_iniciales = [
    #BESTMODELOS
#Mejor KNN: {'weights': 'distance', 'n_neighbors': np.int64(16), 'metric': 'manhattan'}
#Mejor HistGradientBoosting: {'min_samples_leaf': 50, 'max_iter': 300, 'max_depth': 5, 'learning_rate': 0.1, 'l2_regularization': 0.1}
#Mejor RandomForest: {'n_estimators': 200, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_depth': 20, 'bootstrap': True}
#Mejor LGBM: {'subsample': 0.8, 'num_leaves': 50, 'n_estimators': 300, 'min_child_samples': 30, 'max_depth': 6, 'learning_rate': 0.05, 'colsample_bytree': 0.6}
#Mejor CatBoost: {'learning_rate': 0.05, 'l2_leaf_reg': 5, 'iterations': 1000, 'depth': 6}
#Mejor XGB: {'subsample': 1.0, 'reg_lambda': 1.0, 'reg_alpha': 1.0, 'n_estimators': 300, 'max_depth': 4, 'learning_rate': 0.1, 'colsample_bytree': 0.8}

    best_knn,
    best_rf,
    best_hgb,
    best_cat, 
    best_lgbm, 
    best_xgb   
]
'''
    XGBRegressor(n_estimators=200, learning_rate=0.1, max_depth=6, random_state=42),
    LGBMRegressor(n_estimators=200, learning_rate=0.05, random_state=42),
    CatBoostRegressor(iterations=200, learning_rate=0.05, depth=6, verbose=0, random_state=42)'''

#Indicamos cual va a ser el metamodelo a usar
#con este y las iniciales sobre 29950
#metamodelo=LinearRegression()
#metamodelo= MLPRegressor(hidden_layer_sizes=(64, 32), activation='relu',solver='adam', max_iter=2000, early_stopping=True,random_state=42)
metamodelo= best_mlp
#SVR por probar

X_meta_train = np.zeros((len(X_train_scaled), len(modelos_iniciales)))
X_meta_test = np.zeros((len(X_test_scaled), len(modelos_iniciales)))

splits=5
kf = KFold(n_splits=splits, shuffle=True, random_state=42)
modelos_entrenados_finales = []  # Para guardar modelos entrenados finales

for modelo_idx, modelo in enumerate(tqdm(modelos_iniciales, desc="Entrenando modelos...")):
    nombremodel = modelo.__class__.__name__
    tqdm.write(f"Entrenando: {nombremodel}")
    
    predicciones_ext = [None] * len(X_train_scaled)
    predicciones_int = []

    for train_idx, val_idx in kf.split(X_train_scaled):
        X_train_int, y_train_int = X_train_scaled[train_idx], y_train_full.iloc[train_idx]
        X_val = X_train_scaled[val_idx]

        modelo_cv = clone(modelo)
        modelo_cv.fit(X_train_int, y_train_int)

        preds_val = modelo_cv.predict(X_val)
        for idx, pred in zip(val_idx, preds_val):
            predicciones_ext[idx] = pred

        predicciones_int.append(modelo_cv.predict(X_test_scaled))

    for i, pred in enumerate(predicciones_ext):
        X_meta_train[i, modelo_idx] = pred

    avg_test_preds = np.mean(predicciones_int, axis=0)
    for i, pred in enumerate(avg_test_preds):
        X_meta_test[i, modelo_idx] = pred

    modelo_entrenado = clone(modelo)
    modelo_entrenado.fit(X_train_scaled, y_train_full)
    modelos_entrenados_finales.append(modelo_entrenado)


metamodelo.fit(X_meta_train, y_train_full)
y_pred_provisional = metamodelo.predict(X_meta_test)

#mae_metamodelo = mean_absolute_error(y_test, y_pred_provisional)
#print(f"\n\nMAE metamodelo: {mae_metamodelo:.4f}")

#  9. Cargar test.csv 
df_test = pd.read_csv('./data/test.csv', delimiter=',')

# Guardar el ID
id_test = df_test['id']
df_test = df_test.drop(columns=['id'])

#  10. Codificar igual que en entrenamiento 
df_test['eficiencia_enerxetica'] = le.transform(df_test['eficiencia_enerxetica'])

df_test = pd.get_dummies(df_test, columns=[
    'tipo_edificacion',
    'calidade_materiais',
    'cor_favorita_propietario',
    'acceso_transporte_publico',
    'orientacion'
])

# Alinear columnas con X_train
df_test = df_test.reindex(columns=X_train_full.columns, fill_value=0)

# Rellenar NaNs
df_test = df_test.fillna(X_train_full.median(numeric_only=True))

#  11. Normalizar 
X_test_final = scaler.transform(df_test)

predicciones_reales=[]
for modelo in modelos_entrenados_finales:
    predicciones_reales.append(modelo.predict(X_test_final))

stack_predicciones=np.column_stack(predicciones_reales)

#  12. Predecir 
y_pred_final = metamodelo.predict(stack_predicciones)

#  13. Guardar resultados 
resultados = pd.DataFrame({
    'id': id_test,
    'prezo_euros': y_pred_final
})
print('PREDICCIONES GENERADAS')
resultados.to_csv('predicciones_stacking10.csv', index=False)


Entrenando modelos...:   0%|          | 0/6 [00:00<?, ?it/s]

Entrenando: KNeighborsRegressor


Entrenando modelos...:  17%|█▋        | 1/6 [00:04<00:21,  4.23s/it]

Entrenando: RandomForestRegressor


Entrenando modelos...:  33%|███▎      | 2/6 [04:18<10:05, 151.37s/it]

...

Entrenando modelos...:  83%|████████▎ | 5/6 [04:42<00:37, 37.99s/it]

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

Entrenando modelos...: 100%|██████████| 6/6 [04:44<00:00, 47.34s/it]


PREDICCIONES GENERADAS


c:\Users\Anxo\Documents\Python\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


## RELLENO DE LOS NAN CON TAL

In [ ]:
### PRUEBA CON LAS OPTIMIZACIONES DE TODOOS LOS MODELOS
#MAE metamodelo: 28784.8810

#BESTMODELOS
#Mejor KNN: {'weights': 'distance', 'n_neighbors': np.int64(16), 'metric': 'manhattan'}
#Mejor HistGradientBoosting: {'min_samples_leaf': 50, 'max_iter': 300, 'max_depth': 5, 'learning_rate': 0.1, 'l2_regularization': 0.1}
#Mejor RandomForest: {'n_estimators': 200, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_depth': 20, 'bootstrap': True}
#Mejor LGBM: {'subsample': 0.8, 'num_leaves': 50, 'n_estimators': 300, 'min_child_samples': 30, 'max_depth': 6, 'learning_rate': 0.05, 'colsample_bytree': 0.6}
#Mejor CatBoost: {'learning_rate': 0.05, 'l2_leaf_reg': 5, 'iterations': 1000, 'depth': 6}
#Mejor XGB: {'subsample': 1.0, 'reg_lambda': 1.0, 'reg_alpha': 1.0, 'n_estimators': 300, 'max_depth': 4, 'learning_rate': 0.1, 'colsample_bytree': 0.8}


#Preprocesado (sacado del KNN)

#  1. Cargar dataset de entrenamiento 
df = pd.read_csv('./data/train.csv', delimiter=',')

#  2. Guardar y separar target 
target = df['prezo_euros']
X = df.drop(columns=['id', 'prezo_euros'])

#  3. Codificar variables categóricas 
# Label Encoding para eficiencia_enerxetica
le = LabelEncoder()
X['eficiencia_enerxetica'] = le.fit_transform(X['eficiencia_enerxetica'])

# One-Hot Encoding para el resto
X = pd.get_dummies(X, columns=[
    'tipo_edificacion',
    'calidade_materiais',
    'cor_favorita_propietario',
    'acceso_transporte_publico',
    'orientacion'
])

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
imputer = IterativeImputer(random_state=42)

#  4. Rellenar NaNs con mediana 
#X = X.fillna(X.median(numeric_only=True))

#X_train es para entrenar a los modelos iniciales
#X_temp se pasa al segundo split
#X_meta es de donde sacamos las predicciones para el metamodelo
#X_test es X_test
X_train_full, X_test, y_train_full, y_test = train_test_split(X, target, test_size=0.2, random_state=42)
X_train_full = pd.DataFrame(imputer.fit_transform(X_train_full), columns=X_train_full.columns)
X_test = pd.DataFrame(imputer.transform(X_test), columns=X_test.columns)

#  6. Normalizar 
#  6.1 Primero setear el scaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_full)
X_test_scaled = scaler.transform(X_test)

########################################################################################
########################################################################################

#Indicamos cuales van a ser los modelos iniciales a entrenar
modelos_iniciales = [
    best_knn,
    best_rf,
    best_hgb,
    best_cat,
    best_lgbm,
    best_xgb   
]
'''
    XGBRegressor(n_estimators=200, learning_rate=0.1, max_depth=6, random_state=42),
    LGBMRegressor(n_estimators=200, learning_rate=0.05, random_state=42),
    CatBoostRegressor(iterations=200, learning_rate=0.05, depth=6, verbose=0, random_state=42)'''

#Indicamos cual va a ser el metamodelo a usar
#con este y las iniciales sobre 29950
#metamodelo=LinearRegression()
metamodelo= MLPRegressor(hidden_layer_sizes=(64, 32), activation='relu',solver='adam', max_iter=2000, early_stopping=True,random_state=42)
#SVR por probar

X_meta_train = np.zeros((len(X_train_scaled), len(modelos_iniciales)))
X_meta_test = np.zeros((len(X_test_scaled), len(modelos_iniciales)))

splits=5
kf = KFold(n_splits=splits, shuffle=True, random_state=42)
modelos_entrenados_finales = []  # Para guardar modelos entrenados finales

for modelo_idx, modelo in enumerate(tqdm(modelos_iniciales, desc="Entrenando modelos...")):
    nombremodel = modelo.__class__.__name__
    tqdm.write(f"Entrenando: {nombremodel}")
    
    predicciones_ext = [None] * len(X_train_scaled)
    predicciones_int = []

    for train_idx, val_idx in kf.split(X_train_scaled):
        X_train_int, y_train_int = X_train_scaled[train_idx], y_train_full.iloc[train_idx]
        X_val = X_train_scaled[val_idx]

        modelo_cv = clone(modelo)
        modelo_cv.fit(X_train_int, y_train_int)

        preds_val = modelo_cv.predict(X_val)
        for idx, pred in zip(val_idx, preds_val):
            predicciones_ext[idx] = pred

        predicciones_int.append(modelo_cv.predict(X_test_scaled))

    for i, pred in enumerate(predicciones_ext):
        X_meta_train[i, modelo_idx] = pred

    avg_test_preds = np.mean(predicciones_int, axis=0)
    for i, pred in enumerate(avg_test_preds):
        X_meta_test[i, modelo_idx] = pred

    modelo_entrenado = clone(modelo)
    modelo_entrenado.fit(X_train_scaled, y_train_full)
    modelos_entrenados_finales.append(modelo_entrenado)


metamodelo.fit(X_meta_train, y_train_full)
y_pred_provisional = metamodelo.predict(X_meta_test)

mae_metamodelo = mean_absolute_error(y_test, y_pred_provisional)
print(f"\n\nMAE metamodelo: {mae_metamodelo:.4f}")


#  9. Cargar test.csv 
df_test = pd.read_csv('./data/test.csv', delimiter=',')

# Guardar el ID
id_test = df_test['id']
df_test = df_test.drop(columns=['id'])

#  10. Codificar igual que en entrenamiento 
df_test['eficiencia_enerxetica'] = le.transform(df_test['eficiencia_enerxetica'])

df_test = pd.get_dummies(df_test, columns=[
    'tipo_edificacion',
    'calidade_materiais',
    'cor_favorita_propietario',
    'acceso_transporte_publico',
    'orientacion'
])

# Alinear columnas con X_train
df_test = df_test.reindex(columns=X_train_full.columns, fill_value=0)

df_test = pd.DataFrame(imputer.transform(df_test), columns=df_test.columns)


# Rellenar NaNs
#df_test = df_test.fillna(X_train_full.median(numeric_only=True))

#  11. Normalizar 
X_test_final = scaler.transform(df_test)

predicciones_reales=[]
for modelo in modelos_entrenados_finales:
    predicciones_reales.append(modelo.predict(X_test_final))

stack_predicciones=np.column_stack(predicciones_reales)

#  12. Predecir 
y_pred_final = metamodelo.predict(stack_predicciones)

#  13. Guardar resultados 
resultados = pd.DataFrame({
    'id': id_test,
    'prezo_euros': y_pred_final
})
print('PREDICCIONES GENERADAS')
resultados.to_csv('predicciones_stacking5.csv', index=False) # Este csv no sirve para nada

Entrenando modelos...:   0%|          | 0/6 [00:00<?, ?it/s]

Entrenando: KNeighborsRegressor


Entrenando modelos...:  17%|█▋        | 1/6 [00:01<00:05,  1.15s/it]

Entrenando: RandomForestRegressor


Entrenando modelos...:  33%|███▎      | 2/6 [01:57<04:35, 68.76s/it]

...

Entrenando modelos...:  83%|████████▎ | 5/6 [02:16<00:19, 19.45s/it]

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

Entrenando modelos...: 100%|██████████| 6/6 [02:17<00:00, 22.97s/it]




MAE metamodelo: 28268.4175
PREDICCIONES GENERADAS


c:\Users\Anxo\Documents\Python\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


In [ ]:
#### ENTRENANDO CON EL DATASET DE TRAIN COMPLETO ###

#Preprocesado (sacado del KNN)

#  1. Cargar dataset de entrenamiento 
df = pd.read_csv('./data/train.csv', delimiter=',')

#  2. Guardar y separar target 
target = df['prezo_euros']
X = df.drop(columns=['id', 'prezo_euros'])

#  3. Codificar variables categóricas 
# Label Encoding para eficiencia_enerxetica
le = LabelEncoder()
X['eficiencia_enerxetica'] = le.fit_transform(X['eficiencia_enerxetica'])

# One-Hot Encoding para el resto
X = pd.get_dummies(X, columns=[
    'tipo_edificacion',
    'calidade_materiais',
    'cor_favorita_propietario',
    'acceso_transporte_publico',
    'orientacion'
])

#  4. Rellenar NaNs con lo que sea que sea esto 
imputer = IterativeImputer(random_state=42)
X = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)

#X_train es para entrenar a los modelos iniciales
#X_temp se pasa al segundo split
#X_meta es de donde sacamos las predicciones para el metamodelo
#X_test es X_test
#X_train_full, X_test, y_train_full, y_test = train_test_split(X, target, test_size=0.2, random_state=42)
X_train_full = X.copy()
y_train_full = target.copy()

#  6. Normalizar 
#  6.1 Primero setear el scaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_full)
X_test_scaled = scaler.transform(X_train_full)
#X_test_scaled = scaler.transform(X_test)

########################################################################################
########################################################################################

#Indicamos cuales van a ser los modelos iniciales a entrenar
modelos_iniciales = [
    best_knn,
    best_rf,
    best_hgb,
    best_cat, 
    best_lgbm, 
    best_xgb
]
'''
    XGBRegressor(n_estimators=200, learning_rate=0.1, max_depth=6, random_state=42),
    LGBMRegressor(n_estimators=200, learning_rate=0.05, random_state=42),
    CatBoostRegressor(iterations=200, learning_rate=0.05, depth=6, verbose=0, random_state=42)'''

#Indicamos cual va a ser el metamodelo a usar
#con este y las iniciales sobre 29950
#metamodelo=LinearRegression()
metamodelo= MLPRegressor(hidden_layer_sizes=(64, 32), activation='relu',solver='adam', max_iter=2000, early_stopping=True,random_state=42)
#SVR por probar

X_meta_train = np.zeros((len(X_train_scaled), len(modelos_iniciales)))
X_meta_test = np.zeros((len(X_test_scaled), len(modelos_iniciales)))

splits=5
kf = KFold(n_splits=splits, shuffle=True, random_state=42)
modelos_entrenados_finales = []  # Para guardar modelos entrenados finales

for modelo_idx, modelo in enumerate(tqdm(modelos_iniciales, desc="Entrenando modelos...")):
    nombremodel = modelo.__class__.__name__
    tqdm.write(f"Entrenando: {nombremodel}")
    
    predicciones_ext = [None] * len(X_train_scaled)
    predicciones_int = []

    for train_idx, val_idx in kf.split(X_train_scaled):
        X_train_int, y_train_int = X_train_scaled[train_idx], y_train_full.iloc[train_idx]
        X_val = X_train_scaled[val_idx]

        modelo_cv = clone(modelo)
        modelo_cv.fit(X_train_int, y_train_int)

        preds_val = modelo_cv.predict(X_val)
        for idx, pred in zip(val_idx, preds_val):
            predicciones_ext[idx] = pred

        predicciones_int.append(modelo_cv.predict(X_test_scaled))

    for i, pred in enumerate(predicciones_ext):
        X_meta_train[i, modelo_idx] = pred

    avg_test_preds = np.mean(predicciones_int, axis=0)
    for i, pred in enumerate(avg_test_preds):
        X_meta_test[i, modelo_idx] = pred

    modelo_entrenado = clone(modelo)
    modelo_entrenado.fit(X_train_scaled, y_train_full)
    modelos_entrenados_finales.append(modelo_entrenado)


metamodelo.fit(X_meta_train, y_train_full)
y_pred_provisional = metamodelo.predict(X_meta_test)

#mae_metamodelo = mean_absolute_error(y_test, y_pred_provisional)
#print(f"\n\nMAE metamodelo: {mae_metamodelo:.4f}")

#  9. Cargar test.csv 
df_test = pd.read_csv('./data/test.csv', delimiter=',')

# Guardar el ID
id_test = df_test['id']
df_test = df_test.drop(columns=['id'])

#  10. Codificar igual que en entrenamiento 
df_test['eficiencia_enerxetica'] = le.transform(df_test['eficiencia_enerxetica'])

df_test = pd.get_dummies(df_test, columns=[
    'tipo_edificacion',
    'calidade_materiais',
    'cor_favorita_propietario',
    'acceso_transporte_publico',
    'orientacion'
])

# Alinear columnas con X_train
df_test = df_test.reindex(columns=X_train_full.columns, fill_value=0)

# Rellenar NaNs, este es el imputer
df_test = pd.DataFrame(imputer.transform(df_test), columns=df_test.columns)

#  11. Normalizar 
X_test_final = scaler.transform(df_test)

predicciones_reales=[]
for modelo in modelos_entrenados_finales:
    predicciones_reales.append(modelo.predict(X_test_final))

stack_predicciones=np.column_stack(predicciones_reales)

#  12. Predecir 
y_pred_final = metamodelo.predict(stack_predicciones)

#  13. Guardar resultados 
resultados = pd.DataFrame({
    'id': id_test,
    'prezo_euros': y_pred_final
})
print('PREDICCIONES GENERADAS')
resultados.to_csv('predicciones_stacking8_imputer.csv', index=False)


Entrenando modelos...:   0%|          | 0/9 [00:00<?, ?it/s]

Entrenando: KNeighborsRegressor


Entrenando modelos...:  11%|█         | 1/9 [00:04<00:34,  4.32s/it]

Entrenando: RandomForestRegressor


Entrenando modelos...:  11%|█         | 1/9 [00:19<02:39, 19.96s/it]


KeyboardInterrupt: 

In [ ]:
#INTENTO DE METER NUEVOS MODELOS SIENDO ESTOS
'''
ExtraTreesRegressor(
    n_estimators=200,
    max_depth=20,
    min_samples_split=5,
    min_samples_leaf=1,
    bootstrap=True,
    random_state=42),

    Ridge(
    alpha=1.0,
    solver='auto',
    random_state=42),
    
    SVR(
    kernel='rbf',
    C=100,
    epsilon=0.1
    '''
### PRUEBA CON LAS OPTIMIZACIONES DE TODOOS LOS MODELOS
#MAE metamodelo: 28784.8810

#BESTMODELOS
#Mejor KNN: {'weights': 'distance', 'n_neighbors': np.int64(16), 'metric': 'manhattan'}
#Mejor HistGradientBoosting: {'min_samples_leaf': 50, 'max_iter': 300, 'max_depth': 5, 'learning_rate': 0.1, 'l2_regularization': 0.1}
#Mejor RandomForest: {'n_estimators': 200, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_depth': 20, 'bootstrap': True}
#Mejor LGBM: {'subsample': 0.8, 'num_leaves': 50, 'n_estimators': 300, 'min_child_samples': 30, 'max_depth': 6, 'learning_rate': 0.05, 'colsample_bytree': 0.6}
#Mejor CatBoost: {'learning_rate': 0.05, 'l2_leaf_reg': 5, 'iterations': 1000, 'depth': 6}
#Mejor XGB: {'subsample': 1.0, 'reg_lambda': 1.0, 'reg_alpha': 1.0, 'n_estimators': 300, 'max_depth': 4, 'learning_rate': 0.1, 'colsample_bytree': 0.8}


#Preprocesado (sacado del KNN)

#  1. Cargar dataset de entrenamiento 
df = pd.read_csv('./data/train.csv', delimiter=',')#

#  2. Guardar y separar target 
target = df['prezo_euros']
X = df.drop(columns=['id', 'prezo_euros'])

#  3. Codificar variables categóricas 
# Label Encoding para eficiencia_enerxetica
le = LabelEncoder()
X['eficiencia_enerxetica'] = le.fit_transform(X['eficiencia_enerxetica'])

# One-Hot Encoding para el resto
X = pd.get_dummies(X, columns=[
    'tipo_edificacion',
    'calidade_materiais',
    'cor_favorita_propietario',
    'acceso_transporte_publico',
    'orientacion'
])

#  4. Rellenar NaNs con mediana 
X = X.fillna(X.median(numeric_only=True))

#X_train es para entrenar a los modelos iniciales
#X_temp se pasa al segundo split
#X_meta es de donde sacamos las predicciones para el metamodelo
#X_test es X_test
X_train_full, X_test, y_train_full, y_test = train_test_split(X, target, test_size=0.2, random_state=42)

#  6. Normalizar 
#  6.1 Primero setear el scaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_full)
X_test_scaled = scaler.transform(X_test)

########################################################################################
########################################################################################

#Indicamos cuales van a ser los modelos iniciales a entrenar
modelos_iniciales = [
    best_knn,
    best_rf,
    best_hgb,
    best_cat,
    best_lgbm,
    best_xgb,

    ExtraTreesRegressor(
    n_estimators=200,
    max_depth=20,
    min_samples_split=5,
    min_samples_leaf=1,
    bootstrap=True,
    random_state=42),

    Ridge(
    alpha=1.0,
    solver='auto',
    random_state=42),
    
    SVR(
    kernel='rbf',
    C=100,
    epsilon=0.1)
]
'''
    XGBRegressor(n_estimators=200, learning_rate=0.1, max_depth=6, random_state=42),
    LGBMRegressor(n_estimators=200, learning_rate=0.05, random_state=42),
    CatBoostRegressor(iterations=200, learning_rate=0.05, depth=6, verbose=0, random_state=42)'''

#Indicamos cual va a ser el metamodelo a usar
#con este y las iniciales sobre 29950
#metamodelo=LinearRegression()
metamodelo= MLPRegressor(hidden_layer_sizes=(64, 32), activation='relu',solver='adam', max_iter=2000, early_stopping=True,random_state=42)
#SVR por probar

X_meta_train = np.zeros((len(X_train_scaled), len(modelos_iniciales)))
X_meta_test = np.zeros((len(X_test_scaled), len(modelos_iniciales)))

splits=5
kf = KFold(n_splits=splits, shuffle=True, random_state=42)
modelos_entrenados_finales = []  # Para guardar modelos entrenados finales

for modelo_idx, modelo in enumerate(tqdm(modelos_iniciales, desc="Entrenando modelos...")):
    nombremodel = modelo.__class__.__name__
    tqdm.write(f"Entrenando: {nombremodel}")
    
    predicciones_ext = [None] * len(X_train_scaled)
    predicciones_int = []

    for train_idx, val_idx in kf.split(X_train_scaled):
        X_train_int, y_train_int = X_train_scaled[train_idx], y_train_full.iloc[train_idx]
        X_val = X_train_scaled[val_idx]

        modelo_cv = clone(modelo)
        modelo_cv.fit(X_train_int, y_train_int)

        preds_val = modelo_cv.predict(X_val)
        for idx, pred in zip(val_idx, preds_val):
            predicciones_ext[idx] = pred

        predicciones_int.append(modelo_cv.predict(X_test_scaled))

    for i, pred in enumerate(predicciones_ext):
        X_meta_train[i, modelo_idx] = pred

    avg_test_preds = np.mean(predicciones_int, axis=0)
    for i, pred in enumerate(avg_test_preds):
        X_meta_test[i, modelo_idx] = pred

    modelo_entrenado = clone(modelo)
    modelo_entrenado.fit(X_train_scaled, y_train_full)
    modelos_entrenados_finales.append(modelo_entrenado)


metamodelo.fit(X_meta_train, y_train_full)
y_pred_provisional = metamodelo.predict(X_meta_test)

mae_metamodelo = mean_absolute_error(y_test, y_pred_provisional)
print(f"\n\nMAE metamodelo: {mae_metamodelo:.4f}")


#  9. Cargar test.csv 
df_test = pd.read_csv('./data/test.csv', delimiter=',')

# Guardar el ID
id_test = df_test['id']
df_test = df_test.drop(columns=['id'])

#  10. Codificar igual que en entrenamiento 
df_test['eficiencia_enerxetica'] = le.transform(df_test['eficiencia_enerxetica'])

df_test = pd.get_dummies(df_test, columns=[
    'tipo_edificacion',
    'calidade_materiais',
    'cor_favorita_propietario',
    'acceso_transporte_publico',
    'orientacion'
])

# Alinear columnas con X_train
df_test = df_test.reindex(columns=X_train_full.columns, fill_value=0)

# Rellenar NaNs
df_test = df_test.fillna(X_train_full.median(numeric_only=True))

#  11. Normalizar 
X_test_final = scaler.transform(df_test)

predicciones_reales=[]
for modelo in modelos_entrenados_finales:
    predicciones_reales.append(modelo.predict(X_test_final))

stack_predicciones=np.column_stack(predicciones_reales)

#  12. Predecir 
y_pred_final = metamodelo.predict(stack_predicciones)

#  13. Guardar resultados 
resultados = pd.DataFrame({
    'id': id_test,
    'prezo_euros': y_pred_final
})
print('PREDICCIONES GENERADAS')
resultados.to_csv('predicciones_stacking5.csv', index=False) # Este csv no sirve para nada

Entrenando modelos...:   0%|          | 0/9 [00:00<?, ?it/s]

Entrenando: KNeighborsRegressor


Entrenando modelos...:  11%|█         | 1/9 [00:01<00:10,  1.34s/it]

Entrenando: RandomForestRegressor


Entrenando modelos...:  22%|██▏       | 2/9 [01:53<07:45, 66.44s/it]

...

Entrenando: Ridge
Entrenando: SVR


Entrenando modelos...: 100%|██████████| 9/9 [03:35<00:00, 23.91s/it]




MAE metamodelo: 28520.9731


c:\Users\Anxo\Documents\Python\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


PREDICCIONES GENERADAS


In [ ]:
#### ENTRENANDO CON EL DATASET DE TRAIN COMPLETO ###

#Preprocesado (sacado del KNN)

#  1. Cargar dataset de entrenamiento 
df = pd.read_csv('./data/train.csv', delimiter=',')

#  2. Guardar y separar target 
target = df['prezo_euros']
X = df.drop(columns=['id', 'prezo_euros'])

#  3. Codificar variables categóricas 
# Label Encoding para eficiencia_enerxetica
le = LabelEncoder()
X['eficiencia_enerxetica'] = le.fit_transform(X['eficiencia_enerxetica'])

# One-Hot Encoding para el resto
X = pd.get_dummies(X, columns=[
    'tipo_edificacion',
    'calidade_materiais',
    'cor_favorita_propietario',
    'acceso_transporte_publico',
    'orientacion'
])

#  4. Rellenar NaNs con mediana 
X = X.fillna(X.median(numeric_only=True))

#X_train es para entrenar a los modelos iniciales
#X_temp se pasa al segundo split
#X_meta es de donde sacamos las predicciones para el metamodelo
#X_test es X_test
#X_train_full, X_test, y_train_full, y_test = train_test_split(X, target, test_size=0.2, random_state=42)
X_train_full = X.copy()
y_train_full = target.copy()

#  6. Normalizar 
#  6.1 Primero setear el scaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_full)
X_test_scaled = scaler.transform(X_train_full)
#X_test_scaled = scaler.transform(X_test)

########################################################################################
########################################################################################

#Indicamos cuales van a ser los modelos iniciales a entrenar
modelos_iniciales = [
    best_knn,
    best_rf,
    best_hgb,
    best_cat, 
    best_lgbm, 
    best_xgb,
    ExtraTreesRegressor(
    n_estimators=200,
    max_depth=20,
    min_samples_split=5,
    min_samples_leaf=1,
    bootstrap=True,
    random_state=42),

    Ridge(
    alpha=1.0,
    solver='auto',
    random_state=42),
    
    SVR(
    kernel='rbf',
    C=100,
    epsilon=0.1)
]
'''
    XGBRegressor(n_estimators=200, learning_rate=0.1, max_depth=6, random_state=42),
    LGBMRegressor(n_estimators=200, learning_rate=0.05, random_state=42),
    CatBoostRegressor(iterations=200, learning_rate=0.05, depth=6, verbose=0, random_state=42)'''

#Indicamos cual va a ser el metamodelo a usar
#con este y las iniciales sobre 29950
#metamodelo=LinearRegression()
metamodelo= MLPRegressor(hidden_layer_sizes=(64, 32), activation='relu',solver='adam', max_iter=2000, early_stopping=True,random_state=42)
#SVR por probar

X_meta_train = np.zeros((len(X_train_scaled), len(modelos_iniciales)))
X_meta_test = np.zeros((len(X_test_scaled), len(modelos_iniciales)))

splits=5
kf = KFold(n_splits=splits, shuffle=True, random_state=42)
modelos_entrenados_finales = []  # Para guardar modelos entrenados finales

for modelo_idx, modelo in enumerate(tqdm(modelos_iniciales, desc="Entrenando modelos...")):
    nombremodel = modelo.__class__.__name__
    tqdm.write(f"Entrenando: {nombremodel}")
    
    predicciones_ext = [None] * len(X_train_scaled)
    predicciones_int = []

    for train_idx, val_idx in kf.split(X_train_scaled):
        X_train_int, y_train_int = X_train_scaled[train_idx], y_train_full.iloc[train_idx]
        X_val = X_train_scaled[val_idx]

        modelo_cv = clone(modelo)
        modelo_cv.fit(X_train_int, y_train_int)

        preds_val = modelo_cv.predict(X_val)
        for idx, pred in zip(val_idx, preds_val):
            predicciones_ext[idx] = pred

        predicciones_int.append(modelo_cv.predict(X_test_scaled))

    for i, pred in enumerate(predicciones_ext):
        X_meta_train[i, modelo_idx] = pred

    avg_test_preds = np.mean(predicciones_int, axis=0)
    for i, pred in enumerate(avg_test_preds):
        X_meta_test[i, modelo_idx] = pred

    modelo_entrenado = clone(modelo)
    modelo_entrenado.fit(X_train_scaled, y_train_full)
    modelos_entrenados_finales.append(modelo_entrenado)


metamodelo.fit(X_meta_train, y_train_full)
y_pred_provisional = metamodelo.predict(X_meta_test)

#mae_metamodelo = mean_absolute_error(y_test, y_pred_provisional)
#print(f"\n\nMAE metamodelo: {mae_metamodelo:.4f}")

#  9. Cargar test.csv 
df_test = pd.read_csv('./data/test.csv', delimiter=',')

# Guardar el ID
id_test = df_test['id']
df_test = df_test.drop(columns=['id'])

#  10. Codificar igual que en entrenamiento 
df_test['eficiencia_enerxetica'] = le.transform(df_test['eficiencia_enerxetica'])

df_test = pd.get_dummies(df_test, columns=[
    'tipo_edificacion',
    'calidade_materiais',
    'cor_favorita_propietario',
    'acceso_transporte_publico',
    'orientacion'
])

# Alinear columnas con X_train
df_test = df_test.reindex(columns=X_train_full.columns, fill_value=0)

# Rellenar NaNs
df_test = df_test.fillna(X_train_full.median(numeric_only=True))

#  11. Normalizar 
X_test_final = scaler.transform(df_test)

predicciones_reales=[]
for modelo in modelos_entrenados_finales:
    predicciones_reales.append(modelo.predict(X_test_final))

stack_predicciones=np.column_stack(predicciones_reales)

#  12. Predecir 
y_pred_final = metamodelo.predict(stack_predicciones)

#  13. Guardar resultados 
resultados = pd.DataFrame({
    'id': id_test,
    'prezo_euros': y_pred_final
})
print('PREDICCIONES GENERADAS')
resultados.to_csv('predicciones_stacking11.csv', index=False)


Entrenando modelos...:   0%|          | 0/9 [00:00<?, ?it/s]

Entrenando: KNeighborsRegressor


Entrenando modelos...:  11%|█         | 1/9 [00:03<00:30,  3.87s/it]

Entrenando: RandomForestRegressor


Entrenando modelos...:  22%|██▏       | 2/9 [02:29<10:09, 87.07s/it]

...

Entrenando: ExtraTreesRegressor


Entrenando modelos...:  78%|███████▊  | 7/9 [03:38<00:51, 25.69s/it]

Entrenando: Ridge
Entrenando: SVR


Entrenando modelos...: 100%|██████████| 9/9 [05:54<00:00, 39.44s/it]
c:\Users\Anxo\Documents\Python\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


PREDICCIONES GENERADAS


In [ ]:
'''metamodelo = StackingRegressor(
    estimators=[
        ('ridge', Ridge(alpha=1.0)),
        ('svr', SVR(C=100)),
        ('mlp', MLPRegressor(hidden_layer_sizes=(64, 32), max_iter=2000, early_stopping=True, random_state=42))
    ],
    final_estimator=GradientBoostingRegressor(n_estimators=100, learning_rate=0.05, max_depth=4),
    passthrough=True,
    cv=5
)'''

#INTENTO DE METER NUEVOS MODELOS SIENDO ESTOS
'''
ExtraTreesRegressor(
    n_estimators=200,
    max_depth=20,
    min_samples_split=5,
    min_samples_leaf=1,
    bootstrap=True,
    random_state=42),

    Ridge(
    alpha=1.0,
    solver='auto',
    random_state=42),
    
    SVR(
    kernel='rbf',
    C=100,
    epsilon=0.1
    '''
### PRUEBA CON LAS OPTIMIZACIONES DE TODOOS LOS MODELOS
#MAE metamodelo: 28784.8810

#BESTMODELOS
#Mejor KNN: {'weights': 'distance', 'n_neighbors': np.int64(16), 'metric': 'manhattan'}
#Mejor HistGradientBoosting: {'min_samples_leaf': 50, 'max_iter': 300, 'max_depth': 5, 'learning_rate': 0.1, 'l2_regularization': 0.1}
#Mejor RandomForest: {'n_estimators': 200, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_depth': 20, 'bootstrap': True}
#Mejor LGBM: {'subsample': 0.8, 'num_leaves': 50, 'n_estimators': 300, 'min_child_samples': 30, 'max_depth': 6, 'learning_rate': 0.05, 'colsample_bytree': 0.6}
#Mejor CatBoost: {'learning_rate': 0.05, 'l2_leaf_reg': 5, 'iterations': 1000, 'depth': 6}
#Mejor XGB: {'subsample': 1.0, 'reg_lambda': 1.0, 'reg_alpha': 1.0, 'n_estimators': 300, 'max_depth': 4, 'learning_rate': 0.1, 'colsample_bytree': 0.8}


#Preprocesado (sacado del KNN)

#  1. Cargar dataset de entrenamiento 
df = pd.read_csv('./data/train.csv', delimiter=',')#

#  2. Guardar y separar target 
target = df['prezo_euros']
X = df.drop(columns=['id', 'prezo_euros'])

#  3. Codificar variables categóricas 
# Label Encoding para eficiencia_enerxetica
le = LabelEncoder()
X['eficiencia_enerxetica'] = le.fit_transform(X['eficiencia_enerxetica'])

# One-Hot Encoding para el resto
X = pd.get_dummies(X, columns=[
    'tipo_edificacion',
    'calidade_materiais',
    'cor_favorita_propietario',
    'acceso_transporte_publico',
    'orientacion'
])

#  4. Rellenar NaNs con mediana 
X = X.fillna(X.median(numeric_only=True))

#X_train es para entrenar a los modelos iniciales
#X_temp se pasa al segundo split
#X_meta es de donde sacamos las predicciones para el metamodelo
#X_test es X_test
X_train_full, X_test, y_train_full, y_test = train_test_split(X, target, test_size=0.2, random_state=42)

#  6. Normalizar 
#  6.1 Primero setear el scaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_full)
X_test_scaled = scaler.transform(X_test)

########################################################################################
########################################################################################

#Indicamos cuales van a ser los modelos iniciales a entrenar
modelos_iniciales = [
    best_knn,
    best_rf,
    best_hgb,
    best_cat,
    best_lgbm,
    best_xgb,

    ExtraTreesRegressor(
    n_estimators=200,
    max_depth=20,
    min_samples_split=5,
    min_samples_leaf=1,
    bootstrap=True,
    random_state=42),

    Ridge(
    alpha=1.0,
    solver='auto',
    random_state=42),
    
    SVR(
    kernel='rbf',
    C=100,
    epsilon=0.1)
]
'''
    XGBRegressor(n_estimators=200, learning_rate=0.1, max_depth=6, random_state=42),
    LGBMRegressor(n_estimators=200, learning_rate=0.05, random_state=42),
    CatBoostRegressor(iterations=200, learning_rate=0.05, depth=6, verbose=0, random_state=42)'''

#Indicamos cual va a ser el metamodelo a usar
#con este y las iniciales sobre 29950
#metamodelo=LinearRegression()
#SVR por probar
stack_model = StackingRegressor(
    estimators=[
        ('rf', RandomForestRegressor(
            n_estimators=200, max_depth=15, min_samples_split=5, random_state=42)),
        ('knn', KNeighborsRegressor(
            n_neighbors=16, weights='distance', metric='manhattan'))
    ],
    final_estimator=MLPRegressor(
        hidden_layer_sizes=(64, 32),
        activation='relu',
        solver='adam',
        learning_rate='adaptive',
        alpha=1e-4,
        max_iter=2000,
        early_stopping=True,
        random_state=42
    ),
    passthrough=False,  # Only use predictions, not original features
    cv=5,
    n_jobs=-1
)

X_meta_train = np.zeros((len(X_train_scaled), len(modelos_iniciales)))
X_meta_test = np.zeros((len(X_test_scaled), len(modelos_iniciales)))

splits=5
kf = KFold(n_splits=splits, shuffle=True, random_state=42)
modelos_entrenados_finales = []  # Para guardar modelos entrenados finales

for modelo_idx, modelo in enumerate(tqdm(modelos_iniciales, desc="Entrenando modelos...")):
    nombremodel = modelo.__class__.__name__
    tqdm.write(f"Entrenando: {nombremodel}")
    
    predicciones_ext = [None] * len(X_train_scaled)
    predicciones_int = []

    for train_idx, val_idx in kf.split(X_train_scaled):
        X_train_int, y_train_int = X_train_scaled[train_idx], y_train_full.iloc[train_idx]
        X_val = X_train_scaled[val_idx]

        modelo_cv = clone(modelo)
        modelo_cv.fit(X_train_int, y_train_int)

        preds_val = modelo_cv.predict(X_val)
        for idx, pred in zip(val_idx, preds_val):
            predicciones_ext[idx] = pred

        predicciones_int.append(modelo_cv.predict(X_test_scaled))

    for i, pred in enumerate(predicciones_ext):
        X_meta_train[i, modelo_idx] = pred

    avg_test_preds = np.mean(predicciones_int, axis=0)
    for i, pred in enumerate(avg_test_preds):
        X_meta_test[i, modelo_idx] = pred

    modelo_entrenado = clone(modelo)
    modelo_entrenado.fit(X_train_scaled, y_train_full)
    modelos_entrenados_finales.append(modelo_entrenado)


metamodelo.fit(X_meta_train, y_train_full)
y_pred_provisional = metamodelo.predict(X_meta_test)

mae_metamodelo = mean_absolute_error(y_test, y_pred_provisional)
print(f"\n\nMAE metamodelo: {mae_metamodelo:.4f}")


#  9. Cargar test.csv 
df_test = pd.read_csv('./data/test.csv', delimiter=',')

# Guardar el ID
id_test = df_test['id']
df_test = df_test.drop(columns=['id'])

#  10. Codificar igual que en entrenamiento 
df_test['eficiencia_enerxetica'] = le.transform(df_test['eficiencia_enerxetica'])

df_test = pd.get_dummies(df_test, columns=[
    'tipo_edificacion',
    'calidade_materiais',
    'cor_favorita_propietario',
    'acceso_transporte_publico',
    'orientacion'
])

# Alinear columnas con X_train
df_test = df_test.reindex(columns=X_train_full.columns, fill_value=0)

# Rellenar NaNs
df_test = df_test.fillna(X_train_full.median(numeric_only=True))

#  11. Normalizar 
X_test_final = scaler.transform(df_test)

predicciones_reales=[]
for modelo in modelos_entrenados_finales:
    predicciones_reales.append(modelo.predict(X_test_final))

stack_predicciones=np.column_stack(predicciones_reales)

#  12. Predecir 
y_pred_final = metamodelo.predict(stack_predicciones)

#  13. Guardar resultados 
resultados = pd.DataFrame({
    'id': id_test,
    'prezo_euros': y_pred_final
})
print('PREDICCIONES GENERADAS')
resultados.to_csv('predicciones_stacking5.csv', index=False) # Este csv no sirve para nada

Entrenando modelos...:   0%|          | 0/9 [00:00<?, ?it/s]

Entrenando: KNeighborsRegressor


Entrenando modelos...:  11%|█         | 1/9 [00:01<00:09,  1.16s/it]

Entrenando: RandomForestRegressor


Entrenando modelos...:  22%|██▏       | 2/9 [02:55<12:00, 102.94s/it]

...

Entrenando: Ridge
Entrenando: SVR


Entrenando modelos...: 100%|██████████| 9/9 [05:16<00:00, 35.17s/it]




MAE metamodelo: 28648.1785


c:\Users\Anxo\Documents\Python\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


PREDICCIONES GENERADAS


In [ ]:
## MENOS ES MAS
# MAE metamodelo: 28723.4725

#Preprocesado (sacado del KNN)

#  1. Cargar dataset de entrenamiento 
df = pd.read_csv('./data/train.csv', delimiter=',')#

#  2. Guardar y separar target 
target = df['prezo_euros']
X = df.drop(columns=['id', 'prezo_euros'])

#  3. Codificar variables categóricas 
# Label Encoding para eficiencia_enerxetica
le = LabelEncoder()
X['eficiencia_enerxetica'] = le.fit_transform(X['eficiencia_enerxetica'])

# One-Hot Encoding para el resto
X = pd.get_dummies(X, columns=[
    'tipo_edificacion',
    'calidade_materiais',
    'cor_favorita_propietario',
    'acceso_transporte_publico',
    'orientacion'
])

#  4. Rellenar NaNs con mediana 
X = X.fillna(X.median(numeric_only=True))

#X_train es para entrenar a los modelos iniciales
#X_temp se pasa al segundo split
#X_meta es de donde sacamos las predicciones para el metamodelo
#X_test es X_test
X_train_full, X_test, y_train_full, y_test = train_test_split(X, target, test_size=0.2, random_state=42)

#  6. Normalizar 
#  6.1 Primero setear el scaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_full)
X_test_scaled = scaler.transform(X_test)

########################################################################################
########################################################################################

#Indicamos cuales van a ser los modelos iniciales a entrenar
modelos_iniciales = [
    best_hgb,  # HistGradientBoosting: robusto y rápido

    best_lgbm,  # LGBM: muy eficiente con grandes datasets y muchas features

    SVR(C=20, epsilon=0.1, kernel='rbf'),  # SVR: modelo no lineal que aporta diversidad

    Ridge(alpha=10.0),  # Ridge: simple, pero ayuda a la estabilidad del metamodelo

    KNeighborsRegressor(n_neighbors=12, weights='distance', metric='manhattan'),  # KNN: captura relaciones locales

    MLPRegressor(hidden_layer_sizes=(128,), max_iter=2000, early_stopping=True, random_state=42),  # MLP: más profundo que antes

    CatBoostRegressor(iterations=500, learning_rate=0.05, depth=6, verbose=0, random_state=42)  # CatBoost: complementa boosting
]


'''
    XGBRegressor(n_estimators=200, learning_rate=0.1, max_depth=6, random_state=42),
    LGBMRegressor(n_estimators=200, learning_rate=0.05, random_state=42),
    CatBoostRegressor(iterations=200, learning_rate=0.05, depth=6, verbose=0, random_state=42)'''

#Indicamos cual va a ser el metamodelo a usar
#con este y las iniciales sobre 29950
metamodelo=MLPRegressor(hidden_layer_sizes=(64, 32), activation='relu',solver='adam', max_iter=2000, early_stopping=True,random_state=42)
#SVR por probar


X_meta_train = np.zeros((len(X_train_scaled), len(modelos_iniciales)))
X_meta_test = np.zeros((len(X_test_scaled), len(modelos_iniciales)))

splits=5
kf = KFold(n_splits=splits, shuffle=True, random_state=42)
modelos_entrenados_finales = []  # Para guardar modelos entrenados finales

for modelo_idx, modelo in enumerate(tqdm(modelos_iniciales, desc="Entrenando modelos...")):
    nombremodel = modelo.__class__.__name__
    tqdm.write(f"Entrenando: {nombremodel}")
    
    predicciones_ext = [None] * len(X_train_scaled)
    predicciones_int = []

    for train_idx, val_idx in kf.split(X_train_scaled):
        X_train_int, y_train_int = X_train_scaled[train_idx], y_train_full.iloc[train_idx]
        X_val = X_train_scaled[val_idx]

        modelo_cv = clone(modelo)
        modelo_cv.fit(X_train_int, y_train_int)

        preds_val = modelo_cv.predict(X_val)
        for idx, pred in zip(val_idx, preds_val):
            predicciones_ext[idx] = pred

        predicciones_int.append(modelo_cv.predict(X_test_scaled))

    for i, pred in enumerate(predicciones_ext):
        X_meta_train[i, modelo_idx] = pred

    avg_test_preds = np.mean(predicciones_int, axis=0)
    for i, pred in enumerate(avg_test_preds):
        X_meta_test[i, modelo_idx] = pred

    modelo_entrenado = clone(modelo)
    modelo_entrenado.fit(X_train_scaled, y_train_full)
    modelos_entrenados_finales.append(modelo_entrenado)


metamodelo.fit(X_meta_train, y_train_full)
y_pred_provisional = metamodelo.predict(X_meta_test)

mae_metamodelo = mean_absolute_error(y_test, y_pred_provisional)
print(f"\n\nMAE metamodelo: {mae_metamodelo:.4f}")


#  9. Cargar test.csv 
df_test = pd.read_csv('./data/test.csv', delimiter=',')

# Guardar el ID
id_test = df_test['id']
df_test = df_test.drop(columns=['id'])

#  10. Codificar igual que en entrenamiento 
df_test['eficiencia_enerxetica'] = le.transform(df_test['eficiencia_enerxetica'])

df_test = pd.get_dummies(df_test, columns=[
    'tipo_edificacion',
    'calidade_materiais',
    'cor_favorita_propietario',
    'acceso_transporte_publico',
    'orientacion'
])

# Alinear columnas con X_train
df_test = df_test.reindex(columns=X_train_full.columns, fill_value=0)

# Rellenar NaNs
df_test = df_test.fillna(X_train_full.median(numeric_only=True))

#  11. Normalizar 
X_test_final = scaler.transform(df_test)

predicciones_reales=[]
for modelo in modelos_entrenados_finales:
    predicciones_reales.append(modelo.predict(X_test_final))

stack_predicciones=np.column_stack(predicciones_reales)

#  12. Predecir 
y_pred_final = metamodelo.predict(stack_predicciones)

#  13. Guardar resultados 
resultados = pd.DataFrame({
    'id': id_test,
    'prezo_euros': y_pred_final
})
print('PREDICCIONES GENERADAS')
resultados.to_csv('predicciones_stacking5.csv', index=False) # Este csv no sirve para nada

Entrenando modelos...:   0%|          | 0/7 [00:00<?, ?it/s]

Entrenando: HistGradientBoostingRegressor


Entrenando modelos...:  14%|█▍        | 1/7 [00:04<00:27,  4.57s/it]

Entrenando: LGBMRegressor
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002670 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2028
[LightGBM] [Info] Number of data points in the train set: 12800, number of used features: 32
[LightGBM] [Info] Start training from score 223160.343594
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No fur

c:\Users\lucia\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
c:\Users\lucia\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


...

Entrenando: CatBoostRegressor


Entrenando modelos...: 100%|██████████| 7/7 [03:49<00:00, 32.82s/it]




MAE metamodelo: 28723.4725


c:\Users\lucia\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


PREDICCIONES GENERADAS


In [39]:
# REDES NEURONALES


#Preprocesado (sacado del KNN)

#  1. Cargar dataset de entrenamiento 
df = pd.read_csv('./data/train.csv', delimiter=',')#

#  2. Guardar y separar target 
target = df['prezo_euros']
X = df.drop(columns=['id', 'prezo_euros'])

#  3. Codificar variables categóricas 
# Label Encoding para eficiencia_enerxetica
le = LabelEncoder()
X['eficiencia_enerxetica'] = le.fit_transform(X['eficiencia_enerxetica'])

# One-Hot Encoding para el resto
X = pd.get_dummies(X, columns=[
    'tipo_edificacion',
    'calidade_materiais',
    'cor_favorita_propietario',
    'acceso_transporte_publico',
    'orientacion'
])

#  4. Rellenar NaNs con mediana 
X = X.fillna(X.median(numeric_only=True))

#X_train es para entrenar a los modelos iniciales
#X_temp se pasa al segundo split
#X_meta es de donde sacamos las predicciones para el metamodelo
#X_test es X_test
X_train_full, X_test, y_train_full, y_test = train_test_split(X, target, test_size=0.2, random_state=42)

#  6. Normalizar 
#  6.1 Primero setear el scaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_full)
X_test_scaled = scaler.transform(X_test)

########################################################################################
########################################################################################

#Indicamos cuales van a ser los modelos iniciales a entrenar
modelos_iniciales = [
    best_knn,
    best_rf,
    best_hgb,
    best_cat,
    best_lgbm,
    best_xgb,

    ExtraTreesRegressor(
    n_estimators=200,
    max_depth=20,
    min_samples_split=5,
    min_samples_leaf=1,
    bootstrap=True,
    random_state=42),

    Ridge(
    alpha=1.0,
    solver='auto',
    random_state=42),
    
    SVR(
    kernel='rbf',
    C=100,
    epsilon=0.1)
]
'''
    XGBRegressor(n_estimators=200, learning_rate=0.1, max_depth=6, random_state=42),
    LGBMRegressor(n_estimators=200, learning_rate=0.05, random_state=42),
    CatBoostRegressor(iterations=200, learning_rate=0.05, depth=6, verbose=0, random_state=42)'''

#Indicamos cual va a ser el metamodelo a usar
#con este y las iniciales sobre 29950
#metamodelo=LinearRegression()
metamodelo= MLPRegressor(hidden_layer_sizes=(128, 64, 32), activation='relu',solver='adam', max_iter=3000, early_stopping=True,random_state=42)

X_meta_train = np.zeros((len(X_train_scaled), len(modelos_iniciales)))
X_meta_test = np.zeros((len(X_test_scaled), len(modelos_iniciales)))

splits=5
kf = KFold(n_splits=splits, shuffle=True, random_state=42)
modelos_entrenados_finales = []  # Para guardar modelos entrenados finales

for modelo_idx, modelo in enumerate(tqdm(modelos_iniciales, desc="Entrenando modelos...")):
    nombremodel = modelo.__class__.__name__
    tqdm.write(f"Entrenando: {nombremodel}")
    
    predicciones_ext = [None] * len(X_train_scaled)
    predicciones_int = []

    for train_idx, val_idx in kf.split(X_train_scaled):
        X_train_int, y_train_int = X_train_scaled[train_idx], y_train_full.iloc[train_idx]
        X_val = X_train_scaled[val_idx]

        modelo_cv = clone(modelo)
        modelo_cv.fit(X_train_int, y_train_int)

        preds_val = modelo_cv.predict(X_val)
        for idx, pred in zip(val_idx, preds_val):
            predicciones_ext[idx] = pred

        predicciones_int.append(modelo_cv.predict(X_test_scaled))

    for i, pred in enumerate(predicciones_ext):
        X_meta_train[i, modelo_idx] = pred

    avg_test_preds = np.mean(predicciones_int, axis=0)
    for i, pred in enumerate(avg_test_preds):
        X_meta_test[i, modelo_idx] = pred

    modelo_entrenado = clone(modelo)
    modelo_entrenado.fit(X_train_scaled, y_train_full)
    modelos_entrenados_finales.append(modelo_entrenado)


metamodelo.fit(X_meta_train, y_train_full)
y_pred_provisional = metamodelo.predict(X_meta_test)

mae_metamodelo = mean_absolute_error(y_test, y_pred_provisional)
print(f"\n\nMAE metamodelo: {mae_metamodelo:.4f}")


#  9. Cargar test.csv 
df_test = pd.read_csv('./data/test.csv', delimiter=',')

# Guardar el ID
id_test = df_test['id']
df_test = df_test.drop(columns=['id'])

#  10. Codificar igual que en entrenamiento 
df_test['eficiencia_enerxetica'] = le.transform(df_test['eficiencia_enerxetica'])

df_test = pd.get_dummies(df_test, columns=[
    'tipo_edificacion',
    'calidade_materiais',
    'cor_favorita_propietario',
    'acceso_transporte_publico',
    'orientacion'
])

# Alinear columnas con X_train
df_test = df_test.reindex(columns=X_train_full.columns, fill_value=0)

# Rellenar NaNs
df_test = df_test.fillna(X_train_full.median(numeric_only=True))

#  11. Normalizar 
X_test_final = scaler.transform(df_test)

predicciones_reales=[]
for modelo in modelos_entrenados_finales:
    predicciones_reales.append(modelo.predict(X_test_final))

stack_predicciones=np.column_stack(predicciones_reales)

#  12. Predecir 
y_pred_final = metamodelo.predict(stack_predicciones)

#  13. Guardar resultados 
resultados = pd.DataFrame({
    'id': id_test,
    'prezo_euros': y_pred_final
})
print('PREDICCIONES GENERADAS')
resultados.to_csv('predicciones_stacking5.csv', index=False) # Este csv no sirve para nada

Entrenando modelos...:   0%|          | 0/9 [00:00<?, ?it/s]

Entrenando: KNeighborsRegressor


Entrenando modelos...:  11%|█         | 1/9 [00:02<00:22,  2.82s/it]

Entrenando: RandomForestRegressor


Entrenando modelos...:  22%|██▏       | 2/9 [02:02<08:22, 71.81s/it]

...

Entrenando: Ridge
Entrenando: SVR


Entrenando modelos...: 100%|██████████| 9/9 [04:16<00:00, 28.49s/it]




MAE metamodelo: 28447.4623


c:\Users\lucia\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


PREDICCIONES GENERADAS


In [46]:
#### ULTIMA MEJORA DEL MLP ###

#Preprocesado (sacado del KNN)

#  1. Cargar dataset de entrenamiento 
df = pd.read_csv('./data/train.csv', delimiter=',')

#  2. Guardar y separar target 
target = df['prezo_euros']
X = df.drop(columns=['id', 'prezo_euros'])

#  3. Codificar variables categóricas 
# Label Encoding para eficiencia_enerxetica
le = LabelEncoder()
X['eficiencia_enerxetica'] = le.fit_transform(X['eficiencia_enerxetica'])

# One-Hot Encoding para el resto
X = pd.get_dummies(X, columns=[
    'tipo_edificacion',
    'calidade_materiais',
    'cor_favorita_propietario',
    'acceso_transporte_publico',
    'orientacion'
])

#  4. Rellenar NaNs con mediana 
X = X.fillna(X.median(numeric_only=True))

#X_train es para entrenar a los modelos iniciales
#X_temp se pasa al segundo split
#X_meta es de donde sacamos las predicciones para el metamodelo
#X_test es X_test
#X_train_full, X_test, y_train_full, y_test = train_test_split(X, target, test_size=0.2, random_state=42)
X_train_full = X.copy()
y_train_full = target.copy()

#  6. Normalizar 
#  6.1 Primero setear el scaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_full)
X_test_scaled = scaler.transform(X_train_full)
#X_test_scaled = scaler.transform(X_test)

########################################################################################
########################################################################################

#Indicamos cuales van a ser los modelos iniciales a entrenar
modelos_iniciales = [
    best_knn,
    best_rf,
    best_hgb,
    best_cat, 
    best_lgbm, 
    best_xgb,

    ExtraTreesRegressor(
    n_estimators=200,
    max_depth=20,
    min_samples_split=5,
    min_samples_leaf=1,
    bootstrap=True,
    random_state=42),

    Ridge(
    alpha=1.0,
    solver='auto',
    random_state=42),
    
    SVR(
    kernel='rbf',
    C=100,
    epsilon=0.1)
]
'''
    XGBRegressor(n_estimators=200, learning_rate=0.1, max_depth=6, random_state=42),
    LGBMRegressor(n_estimators=200, learning_rate=0.05, random_state=42),
    CatBoostRegressor(iterations=200, learning_rate=0.05, depth=6, verbose=0, random_state=42)'''

#Indicamos cual va a ser el metamodelo a usar
#con este y las iniciales sobre 29950
#metamodelo=LinearRegression()
metamodelo= MLPRegressor(hidden_layer_sizes=(128, 64, 32), activation='relu',solver='adam', max_iter=3000, early_stopping=True,random_state=42)
#SVR por probar

X_meta_train = np.zeros((len(X_train_scaled), len(modelos_iniciales)))
X_meta_test = np.zeros((len(X_test_scaled), len(modelos_iniciales)))

splits=5
kf = KFold(n_splits=splits, shuffle=True, random_state=42)
modelos_entrenados_finales = []  # Para guardar modelos entrenados finales

for modelo_idx, modelo in enumerate(tqdm(modelos_iniciales, desc="Entrenando modelos...")):
    nombremodel = modelo.__class__.__name__
    tqdm.write(f"Entrenando: {nombremodel}")
    
    predicciones_ext = [None] * len(X_train_scaled)
    predicciones_int = []

    for train_idx, val_idx in kf.split(X_train_scaled):
        X_train_int, y_train_int = X_train_scaled[train_idx], y_train_full.iloc[train_idx]
        X_val = X_train_scaled[val_idx]

        modelo_cv = clone(modelo)
        modelo_cv.fit(X_train_int, y_train_int)

        preds_val = modelo_cv.predict(X_val)
        for idx, pred in zip(val_idx, preds_val):
            predicciones_ext[idx] = pred

        predicciones_int.append(modelo_cv.predict(X_test_scaled))

    for i, pred in enumerate(predicciones_ext):
        X_meta_train[i, modelo_idx] = pred

    avg_test_preds = np.mean(predicciones_int, axis=0)
    for i, pred in enumerate(avg_test_preds):
        X_meta_test[i, modelo_idx] = pred

    modelo_entrenado = clone(modelo)
    modelo_entrenado.fit(X_train_scaled, y_train_full)
    modelos_entrenados_finales.append(modelo_entrenado)


metamodelo.fit(X_meta_train, y_train_full)
y_pred_provisional = metamodelo.predict(X_meta_test)

#mae_metamodelo = mean_absolute_error(y_test, y_pred_provisional)
#print(f"\n\nMAE metamodelo: {mae_metamodelo:.4f}")

#  9. Cargar test.csv 
df_test = pd.read_csv('./data/test.csv', delimiter=',')

# Guardar el ID
id_test = df_test['id']
df_test = df_test.drop(columns=['id'])

#  10. Codificar igual que en entrenamiento 
df_test['eficiencia_enerxetica'] = le.transform(df_test['eficiencia_enerxetica'])

df_test = pd.get_dummies(df_test, columns=[
    'tipo_edificacion',
    'calidade_materiais',
    'cor_favorita_propietario',
    'acceso_transporte_publico',
    'orientacion'
])

# Alinear columnas con X_train
df_test = df_test.reindex(columns=X_train_full.columns, fill_value=0)

# Rellenar NaNs
df_test = df_test.fillna(X_train_full.median(numeric_only=True))

#  11. Normalizar 
X_test_final = scaler.transform(df_test)

predicciones_reales=[]
for modelo in modelos_entrenados_finales:
    predicciones_reales.append(modelo.predict(X_test_final))

stack_predicciones=np.column_stack(predicciones_reales)

#  12. Predecir 
y_pred_final = metamodelo.predict(stack_predicciones)

#  13. Guardar resultados 
resultados = pd.DataFrame({
    'id': id_test,
    'prezo_euros': y_pred_final
})
print('PREDICCIONES GENERADAS')
resultados.to_csv('predicciones_stacking12.csv', index=False)


Entrenando modelos...:   0%|          | 0/9 [00:00<?, ?it/s]

Entrenando: KNeighborsRegressor


Entrenando modelos...:  11%|█         | 1/9 [00:10<01:20, 10.01s/it]

Entrenando: RandomForestRegressor


Entrenando modelos...:  22%|██▏       | 2/9 [02:50<11:27, 98.24s/it]

...

Entrenando: ExtraTreesRegressor


Entrenando modelos...:  78%|███████▊  | 7/9 [04:20<01:01, 30.58s/it]

Entrenando: Ridge
Entrenando: SVR


Entrenando modelos...: 100%|██████████| 9/9 [07:14<00:00, 48.32s/it]
c:\Users\lucia\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


PREDICCIONES GENERADAS


In [41]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import ExtraTreesRegressor

# Definir espacio de parámetros para ExtraTreesRegressor
param_grid_et = {
    'n_estimators': [100, 200, 300],  # Número de árboles
    'max_depth': [None, 10, 20, 30],  # Profundidad máxima de los árboles
    'min_samples_split': [2, 5, 10],  # Mínimos ejemplos para dividir un nodo
    'min_samples_leaf': [1, 2, 4],  # Mínimos ejemplos en la hoja
    'bootstrap': [True, False],  # Si se usa bootstrap para el muestreo
}

et_model = ExtraTreesRegressor(random_state=42)

# Realizar búsqueda aleatoria
search_et = RandomizedSearchCV(
    et_model,
    param_distributions=param_grid_et,
    n_iter=10,  # Número de combinaciones aleatorias
    scoring='neg_mean_absolute_error',  # Evaluar con MAE negativo
    cv=3,  # Validación cruzada
    verbose=1,
    random_state=42,
    n_jobs=-1
)

search_et.fit(X_train_scaled, y_train_full)
best_et = search_et.best_estimator_
print("Mejor ExtraTreesRegressor:", search_et.best_params_)

from sklearn.linear_model import Ridge

# Definir espacio de parámetros para Ridge
param_grid_ridge = {
    'alpha': [0.1, 1.0, 10.0, 100.0],  # Regularización L2
    'solver': ['auto', 'svd', 'cholesky', 'lsqr', 'saga'],  # Método de solución
}

ridge_model = Ridge(random_state=42)

# Realizar búsqueda aleatoria
search_ridge = RandomizedSearchCV(
    ridge_model,
    param_distributions=param_grid_ridge,
    n_iter=10,  # Número de combinaciones aleatorias
    scoring='neg_mean_absolute_error',  # Evaluar con MAE negativo
    cv=3,  # Validación cruzada
    verbose=1,
    random_state=42,
    n_jobs=-1
)

search_ridge.fit(X_train_scaled, y_train_full)
best_ridge = search_ridge.best_estimator_
print("Mejor Ridge:", search_ridge.best_params_)

from sklearn.svm import SVR

# Definir espacio de parámetros para SVR
param_grid_svr = {
    'C': [0.1, 1, 10, 100],  # Penalización de los errores
    'epsilon': [0.01, 0.1, 0.2, 0.5],  # Tolerancia para los márgenes
    'kernel': ['rbf', 'linear', 'poly'],  # Función del kernel
    'gamma': ['scale', 'auto'],  # Parámetro de los kernels no lineales
}

svr_model = SVR()

# Realizar búsqueda aleatoria
search_svr = RandomizedSearchCV(
    svr_model,
    param_distributions=param_grid_svr,
    n_iter=10,  # Número de combinaciones aleatorias
    scoring='neg_mean_absolute_error',  # Evaluar con MAE negativo
    cv=3,  # Validación cruzada
    verbose=1,
    random_state=42,
    n_jobs=-1
)

search_svr.fit(X_train_scaled, y_train_full)
best_svr = search_svr.best_estimator_
print("Mejor SVR:", search_svr.best_params_)


Fitting 3 folds for each of 10 candidates, totalling 30 fits
Mejor ExtraTreesRegressor: {'n_estimators': 100, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_depth': 30, 'bootstrap': False}
Fitting 3 folds for each of 10 candidates, totalling 30 fits
Mejor Ridge: {'solver': 'auto', 'alpha': 0.1}
Fitting 3 folds for each of 10 candidates, totalling 30 fits
Mejor SVR: {'kernel': 'linear', 'gamma': 'scale', 'epsilon': 0.01, 'C': 100}


In [44]:
best_et = ExtraTreesRegressor(
    n_estimators=100,
    max_depth=30,
    min_samples_split=5,
    min_samples_leaf=2,
    bootstrap=False,
    random_state=42
)

best_ridge = Ridge(alpha=0.1, solver='auto', random_state=42)

best_svr = SVR(kernel='linear', gamma='scale', epsilon=0.01, C=100)


In [45]:

#Preprocesado (sacado del KNN)

#  1. Cargar dataset de entrenamiento 
df = pd.read_csv('./data/train.csv', delimiter=',')#

#  2. Guardar y separar target 
target = df['prezo_euros']
X = df.drop(columns=['id', 'prezo_euros'])

#  3. Codificar variables categóricas 
# Label Encoding para eficiencia_enerxetica
le = LabelEncoder()
X['eficiencia_enerxetica'] = le.fit_transform(X['eficiencia_enerxetica'])

# One-Hot Encoding para el resto
X = pd.get_dummies(X, columns=[
    'tipo_edificacion',
    'calidade_materiais',
    'cor_favorita_propietario',
    'acceso_transporte_publico',
    'orientacion'
])

#  4. Rellenar NaNs con mediana 
X = X.fillna(X.median(numeric_only=True))

#X_train es para entrenar a los modelos iniciales
#X_temp se pasa al segundo split
#X_meta es de donde sacamos las predicciones para el metamodelo
#X_test es X_test
X_train_full, X_test, y_train_full, y_test = train_test_split(X, target, test_size=0.2, random_state=42)

#  6. Normalizar 
#  6.1 Primero setear el scaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_full)
X_test_scaled = scaler.transform(X_test)

########################################################################################
########################################################################################

#Indicamos cuales van a ser los modelos iniciales a entrenar
modelos_iniciales = [
    best_knn,
    best_rf,
    best_hgb,
    best_cat,
    best_lgbm,
    best_xgb,
    best_svr,
    best_et,
    best_ridge
]
'''
    XGBRegressor(n_estimators=200, learning_rate=0.1, max_depth=6, random_state=42),
    LGBMRegressor(n_estimators=200, learning_rate=0.05, random_state=42),
    CatBoostRegressor(iterations=200, learning_rate=0.05, depth=6, verbose=0, random_state=42)'''

#Indicamos cual va a ser el metamodelo a usar
#con este y las iniciales sobre 29950
#metamodelo=LinearRegression()
metamodelo= MLPRegressor(hidden_layer_sizes=(128, 64, 32), activation='relu',solver='adam', max_iter=3000, early_stopping=True,random_state=42)

X_meta_train = np.zeros((len(X_train_scaled), len(modelos_iniciales)))
X_meta_test = np.zeros((len(X_test_scaled), len(modelos_iniciales)))

splits=5
kf = KFold(n_splits=splits, shuffle=True, random_state=42)
modelos_entrenados_finales = []  # Para guardar modelos entrenados finales

for modelo_idx, modelo in enumerate(tqdm(modelos_iniciales, desc="Entrenando modelos...")):
    nombremodel = modelo.__class__.__name__
    tqdm.write(f"Entrenando: {nombremodel}")
    
    predicciones_ext = [None] * len(X_train_scaled)
    predicciones_int = []

    for train_idx, val_idx in kf.split(X_train_scaled):
        X_train_int, y_train_int = X_train_scaled[train_idx], y_train_full.iloc[train_idx]
        X_val = X_train_scaled[val_idx]

        modelo_cv = clone(modelo)
        modelo_cv.fit(X_train_int, y_train_int)

        preds_val = modelo_cv.predict(X_val)
        for idx, pred in zip(val_idx, preds_val):
            predicciones_ext[idx] = pred

        predicciones_int.append(modelo_cv.predict(X_test_scaled))

    for i, pred in enumerate(predicciones_ext):
        X_meta_train[i, modelo_idx] = pred

    avg_test_preds = np.mean(predicciones_int, axis=0)
    for i, pred in enumerate(avg_test_preds):
        X_meta_test[i, modelo_idx] = pred

    modelo_entrenado = clone(modelo)
    modelo_entrenado.fit(X_train_scaled, y_train_full)
    modelos_entrenados_finales.append(modelo_entrenado)


metamodelo.fit(X_meta_train, y_train_full)
y_pred_provisional = metamodelo.predict(X_meta_test)

mae_metamodelo = mean_absolute_error(y_test, y_pred_provisional)
print(f"\n\nMAE metamodelo: {mae_metamodelo:.4f}")


#  9. Cargar test.csv 
df_test = pd.read_csv('./data/test.csv', delimiter=',')

# Guardar el ID
id_test = df_test['id']
df_test = df_test.drop(columns=['id'])

#  10. Codificar igual que en entrenamiento 
df_test['eficiencia_enerxetica'] = le.transform(df_test['eficiencia_enerxetica'])

df_test = pd.get_dummies(df_test, columns=[
    'tipo_edificacion',
    'calidade_materiais',
    'cor_favorita_propietario',
    'acceso_transporte_publico',
    'orientacion'
])

# Alinear columnas con X_train
df_test = df_test.reindex(columns=X_train_full.columns, fill_value=0)

# Rellenar NaNs
df_test = df_test.fillna(X_train_full.median(numeric_only=True))

#  11. Normalizar 
X_test_final = scaler.transform(df_test)

predicciones_reales=[]
for modelo in modelos_entrenados_finales:
    predicciones_reales.append(modelo.predict(X_test_final))

stack_predicciones=np.column_stack(predicciones_reales)

#  12. Predecir 
y_pred_final = metamodelo.predict(stack_predicciones)

#  13. Guardar resultados 
resultados = pd.DataFrame({
    'id': id_test,
    'prezo_euros': y_pred_final
})
print('PREDICCIONES GENERADAS')
resultados.to_csv('predicciones_stacking5.csv', index=False) # Este csv no sirve para nada

Entrenando modelos...:   0%|          | 0/9 [00:00<?, ?it/s]

Entrenando: KNeighborsRegressor


Entrenando modelos...:  11%|█         | 1/9 [00:03<00:26,  3.36s/it]

Entrenando: RandomForestRegressor


Entrenando modelos...:  22%|██▏       | 2/9 [02:03<08:22, 71.80s/it]

...

Entrenando: ExtraTreesRegressor


Entrenando modelos...: 100%|██████████| 9/9 [03:33<00:00, 23.71s/it]


Entrenando: Ridge


MAE metamodelo: 28598.8744


c:\Users\lucia\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


PREDICCIONES GENERADAS


In [25]:
### AÑADIMOS CNN

X_train_cnn = X_train_scaled.reshape((X_train_scaled.shape[0], X_train_scaled.shape[1], 1))
X_test_cnn = X_test_scaled.reshape((X_test_scaled.shape[0], X_test_scaled.shape[1], 1))


from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense, MaxPooling1D
from scikeras.wrappers import KerasRegressor 

from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.layers import Dropout

def crear_modelo_cnn():
    model = Sequential([
        # Primera capa convolucional
        Conv1D(64, kernel_size=3, activation='relu', input_shape=(X_train_scaled.shape[1], 1)),
        MaxPooling1D(pool_size=2),
        Dropout(0.2),  # Dropout para evitar el sobreajuste
        
        # Segunda capa convolucional
        Conv1D(128, kernel_size=3, activation='relu'),
        MaxPooling1D(pool_size=2),
        Dropout(0.3),

        # Tercera capa convolucional
        Conv1D(256, kernel_size=3, activation='relu'),
        MaxPooling1D(pool_size=2),
        Dropout(0.4),

        # Aplanado para pasar a la capa densa
        Flatten(),
        
        # Capa densa con regularización L2
        Dense(128, activation='relu', kernel_regularizer=l2(0.01)),  # L2 regularization
        
        # Capa de salida
        Dense(1)
    ])
    
    # Compilación del modelo con un optimizador Adam y tasa de aprendizaje ajustada
    optimizer = Adam(learning_rate=0.001)
    model.compile(optimizer=optimizer, loss='mean_absolute_error')
    
    return model


modelo_cnn = KerasRegressor(build_fn=crear_modelo_cnn, epochs=50, batch_size=32, verbose=0)






####################################3333


# REDES NEURONALES


#Preprocesado (sacado del KNN)

#  1. Cargar dataset de entrenamiento 
df = pd.read_csv('./data/train.csv', delimiter=',')#

#  2. Guardar y separar target 
target = df['prezo_euros']
X = df.drop(columns=['id', 'prezo_euros'])

#  3. Codificar variables categóricas 
# Label Encoding para eficiencia_enerxetica
le = LabelEncoder()
X['eficiencia_enerxetica'] = le.fit_transform(X['eficiencia_enerxetica'])

# One-Hot Encoding para el resto
X = pd.get_dummies(X, columns=[
    'tipo_edificacion',
    'calidade_materiais',
    'cor_favorita_propietario',
    'acceso_transporte_publico',
    'orientacion'
])

#  4. Rellenar NaNs con mediana 
X = X.fillna(X.median(numeric_only=True))

#X_train es para entrenar a los modelos iniciales
#X_temp se pasa al segundo split
#X_meta es de donde sacamos las predicciones para el metamodelo
#X_test es X_test
X_train_full, X_test, y_train_full, y_test = train_test_split(X, target, test_size=0.2, random_state=42)

#  6. Normalizar 
#  6.1 Primero setear el scaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_full)
X_test_scaled = scaler.transform(X_test)

########################################################################################
########################################################################################

#Indicamos cuales van a ser los modelos iniciales a entrenar
modelos_iniciales = [
    best_knn,
    best_rf,
    best_hgb,
    best_cat, 
    best_lgbm, 
    best_xgb,
    modelo_cnn,

    ExtraTreesRegressor(
    n_estimators=200,
    max_depth=20,
    min_samples_split=5,
    min_samples_leaf=1,
    bootstrap=True,
    random_state=42),

    Ridge(
    alpha=1.0,
    solver='auto',
    random_state=42),
    
    SVR(
    kernel='rbf',
    C=100,
    epsilon=0.1)
]
'''
    XGBRegressor(n_estimators=200, learning_rate=0.1, max_depth=6, random_state=42),
    LGBMRegressor(n_estimators=200, learning_rate=0.05, random_state=42),
    CatBoostRegressor(iterations=200, learning_rate=0.05, depth=6, verbose=0, random_state=42)'''

#Indicamos cual va a ser el metamodelo a usar
#con este y las iniciales sobre 29950
#metamodelo=LinearRegression()
metamodelo= MLPRegressor(hidden_layer_sizes=(128, 64, 32), activation='relu',solver='adam', max_iter=3000, early_stopping=True,random_state=42)

X_meta_train = np.zeros((len(X_train_scaled), len(modelos_iniciales)))
X_meta_test = np.zeros((len(X_test_scaled), len(modelos_iniciales)))

splits=5
kf = KFold(n_splits=splits, shuffle=True, random_state=42)
modelos_entrenados_finales = []  # Para guardar modelos entrenados finales

for modelo_idx, modelo in enumerate(tqdm(modelos_iniciales, desc="Entrenando modelos...")):
    nombremodel = modelo.__class__.__name__
    tqdm.write(f"Entrenando: {nombremodel}")
    
    predicciones_ext = [None] * len(X_train_scaled)
    predicciones_int = []

    for train_idx, val_idx in kf.split(X_train_scaled):
        X_train_int, y_train_int = X_train_scaled[train_idx], y_train_full.iloc[train_idx]
        X_val = X_train_scaled[val_idx]

        modelo_cv = clone(modelo)
        modelo_cv.fit(X_train_int, y_train_int)

        preds_val = modelo_cv.predict(X_val)
        for idx, pred in zip(val_idx, preds_val):
            predicciones_ext[idx] = pred

        predicciones_int.append(modelo_cv.predict(X_test_scaled))

    for i, pred in enumerate(predicciones_ext):
        X_meta_train[i, modelo_idx] = pred

    avg_test_preds = np.mean(predicciones_int, axis=0)
    for i, pred in enumerate(avg_test_preds):
        X_meta_test[i, modelo_idx] = pred

    modelo_entrenado = clone(modelo)
    modelo_entrenado.fit(X_train_scaled, y_train_full)
    modelos_entrenados_finales.append(modelo_entrenado)


metamodelo.fit(X_meta_train, y_train_full)
y_pred_provisional = metamodelo.predict(X_meta_test)

mae_metamodelo = mean_absolute_error(y_test, y_pred_provisional)
print(f"\n\nMAE metamodelo: {mae_metamodelo:.4f}")


#  9. Cargar test.csv 
df_test = pd.read_csv('./data/test.csv', delimiter=',')

# Guardar el ID
id_test = df_test['id']
df_test = df_test.drop(columns=['id'])

#  10. Codificar igual que en entrenamiento 
df_test['eficiencia_enerxetica'] = le.transform(df_test['eficiencia_enerxetica'])

df_test = pd.get_dummies(df_test, columns=[
    'tipo_edificacion',
    'calidade_materiais',
    'cor_favorita_propietario',
    'acceso_transporte_publico',
    'orientacion'
])

# Alinear columnas con X_train
df_test = df_test.reindex(columns=X_train_full.columns, fill_value=0)

# Rellenar NaNs
df_test = df_test.fillna(X_train_full.median(numeric_only=True))

#  11. Normalizar 
X_test_final = scaler.transform(df_test)

predicciones_reales=[]
for modelo in modelos_entrenados_finales:
    predicciones_reales.append(modelo.predict(X_test_final))

stack_predicciones=np.column_stack(predicciones_reales)

#  12. Predecir 
y_pred_final = metamodelo.predict(stack_predicciones)

#  13. Guardar resultados 
resultados = pd.DataFrame({
    'id': id_test,
    'prezo_euros': y_pred_final
})
print('PREDICCIONES GENERADAS')
resultados.to_csv('predicciones_stacking5.csv', index=False) # Este csv no sirve para nada

Entrenando modelos...:   0%|          | 0/10 [00:00<?, ?it/s]

Entrenando: KNeighborsRegressor


Entrenando modelos...:  10%|█         | 1/10 [00:03<00:33,  3.72s/it]

Entrenando: RandomForestRegressor


Entrenando modelos...:  20%|██        | 2/10 [02:33<11:57, 89.74s/it]

...

Entrenando: Ridge
Entrenando: SVR


Entrenando modelos...: 100%|██████████| 10/10 [16:26<00:00, 98.67s/it] 




MAE metamodelo: 28566.2670


c:\Users\lucia\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


PREDICCIONES GENERADAS


In [ ]:
#### ULTIMA MEJORA DEL MLP ###

#Preprocesado (sacado del KNN)

#  1. Cargar dataset de entrenamiento 
df = pd.read_csv('./data/train.csv', delimiter=',')

#  2. Guardar y separar target 
target = df['prezo_euros']
X = df.drop(columns=['id', 'prezo_euros'])

#  3. Codificar variables categóricas 
# Label Encoding para eficiencia_enerxetica
le = LabelEncoder()
X['eficiencia_enerxetica'] = le.fit_transform(X['eficiencia_enerxetica'])

# One-Hot Encoding para el resto
X = pd.get_dummies(X, columns=[
    'tipo_edificacion',
    'calidade_materiais',
    'cor_favorita_propietario',
    'acceso_transporte_publico',
    'orientacion'
])

#  4. Rellenar NaNs con mediana 
X = X.fillna(X.median(numeric_only=True))

#X_train es para entrenar a los modelos iniciales
#X_temp se pasa al segundo split
#X_meta es de donde sacamos las predicciones para el metamodelo
#X_test es X_test
#X_train_full, X_test, y_train_full, y_test = train_test_split(X, target, test_size=0.2, random_state=42)
X_train_full = X.copy()
y_train_full = target.copy()

#  6. Normalizar 
#  6.1 Primero setear el scaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_full)
X_test_scaled = scaler.transform(X_train_full)
#X_test_scaled = scaler.transform(X_test)

########################################################################################
########################################################################################

#Indicamos cuales van a ser los modelos iniciales a entrenar
modelos_iniciales = [
    best_knn,
    best_rf,
    best_hgb,
    best_cat, 
    best_lgbm, 
    best_xgb,
    modelo_cnn,

    ExtraTreesRegressor(
    n_estimators=200,
    max_depth=20,
    min_samples_split=5,
    min_samples_leaf=1,
    bootstrap=True,
    random_state=42),

    Ridge(
    alpha=1.0,
    solver='auto',
    random_state=42),
    
    SVR(
    kernel='rbf',
    C=100,
    epsilon=0.1)
]
'''
    XGBRegressor(n_estimators=200, learning_rate=0.1, max_depth=6, random_state=42),
    LGBMRegressor(n_estimators=200, learning_rate=0.05, random_state=42),
    CatBoostRegressor(iterations=200, learning_rate=0.05, depth=6, verbose=0, random_state=42)'''

#Indicamos cual va a ser el metamodelo a usar
#con este y las iniciales sobre 29950
#metamodelo=LinearRegression()
metamodelo= MLPRegressor(hidden_layer_sizes=(128, 64, 32), activation='relu',solver='adam', max_iter=3000, early_stopping=True,random_state=42)
#SVR por probar

X_meta_train = np.zeros((len(X_train_scaled), len(modelos_iniciales)))
X_meta_test = np.zeros((len(X_test_scaled), len(modelos_iniciales)))

splits=5
kf = KFold(n_splits=splits, shuffle=True, random_state=42)
modelos_entrenados_finales = []  # Para guardar modelos entrenados finales

for modelo_idx, modelo in enumerate(tqdm(modelos_iniciales, desc="Entrenando modelos...")):
    nombremodel = modelo.__class__.__name__
    tqdm.write(f"Entrenando: {nombremodel}")
    
    predicciones_ext = [None] * len(X_train_scaled)
    predicciones_int = []

    for train_idx, val_idx in kf.split(X_train_scaled):
        X_train_int, y_train_int = X_train_scaled[train_idx], y_train_full.iloc[train_idx]
        X_val = X_train_scaled[val_idx]

        modelo_cv = clone(modelo)
        modelo_cv.fit(X_train_int, y_train_int)

        preds_val = modelo_cv.predict(X_val)
        for idx, pred in zip(val_idx, preds_val):
            predicciones_ext[idx] = pred

        predicciones_int.append(modelo_cv.predict(X_test_scaled))

    for i, pred in enumerate(predicciones_ext):
        X_meta_train[i, modelo_idx] = pred

    avg_test_preds = np.mean(predicciones_int, axis=0)
    for i, pred in enumerate(avg_test_preds):
        X_meta_test[i, modelo_idx] = pred

    modelo_entrenado = clone(modelo)
    modelo_entrenado.fit(X_train_scaled, y_train_full)
    modelos_entrenados_finales.append(modelo_entrenado)


metamodelo.fit(X_meta_train, y_train_full)
y_pred_provisional = metamodelo.predict(X_meta_test)

#mae_metamodelo = mean_absolute_error(y_test, y_pred_provisional)
#print(f"\n\nMAE metamodelo: {mae_metamodelo:.4f}")

#  9. Cargar test.csv 
df_test = pd.read_csv('./data/test.csv', delimiter=',')

# Guardar el ID
id_test = df_test['id']
df_test = df_test.drop(columns=['id'])

#  10. Codificar igual que en entrenamiento 
df_test['eficiencia_enerxetica'] = le.transform(df_test['eficiencia_enerxetica'])

df_test = pd.get_dummies(df_test, columns=[
    'tipo_edificacion',
    'calidade_materiais',
    'cor_favorita_propietario',
    'acceso_transporte_publico',
    'orientacion'
])

# Alinear columnas con X_train
df_test = df_test.reindex(columns=X_train_full.columns, fill_value=0)

# Rellenar NaNs
df_test = df_test.fillna(X_train_full.median(numeric_only=True))

#  11. Normalizar 
X_test_final = scaler.transform(df_test)

predicciones_reales=[]
for modelo in modelos_entrenados_finales:
    predicciones_reales.append(modelo.predict(X_test_final))

stack_predicciones=np.column_stack(predicciones_reales)

#  12. Predecir 
y_pred_final = metamodelo.predict(stack_predicciones)

#  13. Guardar resultados 
resultados = pd.DataFrame({
    'id': id_test,
    'prezo_euros': y_pred_final
})
print('PREDICCIONES GENERADAS')
resultados.to_csv('predicciones_stacking13.csv', index=False)


## Resultados

## Conclusión 